# Notebook de VARIABLES

Este notebook sirve para probar la clasificación de variables de manera automática en base al documento de **Protocolo de Variables**.

## 0a. Ollama

In [12]:
import ollama
import re
from typing import Optional, List, Any, Dict
import json

def consultar_ollama(
    prompt: str, 
    modelo: str = "gemma3:4b", 
    formato: str = "json",  # Forzamos JSON por defecto
    temperature: float = 0.1 # Bajo para ser preciso/determinista
) -> str:
    """
    Envía un prompt a Ollama con soporte para formato JSON y temperatura.
    """
    try:
        # Configuramos las opciones avanzadas
        opciones = {
            'temperature': temperature,
            'num_ctx': 8192,       # <--- ESTO ES LO IMPORTANTE. Ampliamos la memoria a ~32k caracteres.
            'num_predict': 1024,   # Reservamos espacio suficiente para la respuesta JSON
        }

        response = ollama.chat(
            model=modelo,
            messages=[{'role': 'user', 'content': prompt}],
            format=formato,  # Esto es CLAVE: fuerza al modelo a devolver JSON estructura
            options=opciones
        )
        return response['message']['content'].strip()
    
    except Exception as e:
        print(f"Error conectando con el modelo {modelo}: {e}")
        return ""

## 0b. Cargar datos

In [3]:
import newspaper
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, Field, ValidationError


SUFFIX = "scrape"
data = pd.read_csv(f"../data/2026_02_10_imio_def_todo_envio_heidy.xlsx - 2026_02_09_imio_def_todo_clara_{SUFFIX}.csv")
# data = pd.read_csv("../data/Base de datos Noticias 27102025.xlsx - Noticias_scrape.csv")


data

,no_N_noticia,IdNoticia,Medio_num,Fecha,año,año_agrupado,Caracteres,Titular,nombre_propio_titular,cita_en_titulo,...,no_MES,no_Contenido,no_Pagina_url,no_verificacion,no_textonoticia,antiguo_genero_protagonistas,emocion_ia,IA_razonamiento_titulares,no_vacio_numero_caracteres,contenido_articulo
0,1,1,2,01/01/2024,2024,2,4515,el papa denuncia la violencia contra las mujer...,2,2,...,1.0,"“quien lastima a una mujer profana a dios”, ha...",https://elpais.com/sociedad/2024-01-01/el-papa...,1.0,el papa francisco ha denunciado la violencia c...,3,5.0,El Papa (hombre),NaN,El papa Francisco ha denunciado la violencia c...
1,2,2,2,01/01/2024,2024,2,4880,"inteligencia artificial: agárrense, que vienen...",1,1,...,1.0,los despidos masivos de una industria tan rent...,https://elpais.com/babelia/2024-01-01/intelige...,1.0,"baldur’s gate 3, zelda: tears of the kingdom, ...",4,7.0,NaN,NaN,Artículos estrictamente de opinión que respond...
2,3,3,2,01/01/2024,2024,2,9052,en 2024 votarán miles de millones de personas ...,1,1,...,1.0,la concentración de procesos electorales en un...,https://elpais.com/tecnologia/2024-01-01/en-20...,1.0,"durante el año 2024, el calendario electoral s...",1,7.0,NaN,NaN,Una persona antes de coger una papeleta para e...
3,4,4,2,02/01/2024,2024,2,2523,estupidez artificial,1,1,...,1.0,lo tenebroso no son las nuevas formas de traba...,https://elpais.com/opinion/2024-01-02/estupide...,1.0,lo que da miedo de 2024 no es la inteligencia ...,1,6.0,NaN,NaN,Lo tenebroso no son las nuevas formas de traba...
4,5,5,2,02/01/2024,2024,2,3177,la última navidad,1,1,...,1.0,los lectores escriben sobre la ilusión de los ...,https://elpais.com/opinion/2024-01-02/la-ultim...,1.0,estoy triste. sé positivamente que esta ha sid...,1,3.0,NaN,NaN,Los lectores escriben sobre la ilusión de los ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7110,7111,j995,7,18/01/2017,2018,1,1631,Un ‘pezrobot’ tan realista que puede filmar pe...,1,1,...,NaN,x,https://www.lavanguardia.com/natural/20180322/...,NaN,NaN,4,NaN,No hay.,NaN,La mayoría de los peces robot no se encuentran...
7111,7112,j996,7,18/01/2017,2018,1,2553,YouTube prioriza contenido extremista,5,1,...,NaN,x,https://www.lavanguardia.com/tecnologia/201803...,NaN,NaN,3,NaN,YouTube (empresas).,NaN,NaN
7112,7113,j997,7,05/01/2017,2018,1,4599,Estos son los retos del futuro de la automoción,1,1,...,NaN,x,https://www.lavanguardia.com/motor/20180226/44...,NaN,NaN,4,NaN,No hay.,NaN,El futuro del coche conectado en Volkswagen es...
7113,7114,j998,1,18/01/2017,2021,1,6479,"Lucía Velasco, tecnóloga: ""Hay 15s que recomie...",3,2,...,NaN,x,https://www.elmundo.es/yodona/lifestyle/2021/1...,NaN,NaN,2,NaN,Lucía Velasco (mujer).,NaN,Tú crees que eso de los algoritmos no va conti...


### Artículo

In [4]:
articulo_raw = data[data['IdNoticia'] == "1"].iloc[0]
print(articulo_raw)

# articulo_raw = data[data['IdNoticia'] == 46].iloc[0]
# print(articulo_raw)

no_N_noticia                                                                         1
IdNoticia                                                                            1
Medio_num                                                                            2
Fecha                                                                       01/01/2024
año                                                                               2024
año_agrupado                                                                         2
Caracteres                                                                        4515
Titular                              el papa denuncia la violencia contra las mujer...
nombre_propio_titular                                                                2
cita_en_titulo                                                                       2
cla_genero_prota                                                                     3
nombre_periodista                          

#### Texto del artículo

In [5]:
# articulo_text = articulo['no_Contenido']
articulo_text = articulo_raw['contenido_articulo']

articulo_text

'El papa Francisco ha denunciado la violencia contra las mujeres en la misa del día de Año Nuevo, y ha defendido que la Iglesia debe darle más espacio a ellas y “redescubrir su rostro femenino”. “Quien lastima a una mujer profana a Dios, nacido de mujer”, condenó Bergoglio. Y recordó: “Toda sociedad necesita acoger el don de la mujer, de cada mujer: respetarla, cuidarla, valorarla”.\n\nEl día 1 de enero el cristianismo dedica el día a la Virgen, por eso el Pontífice abrió el año con una misa en la basílica en la que ensalzó el papel de María y de la mujer en la Iglesia. Francisco hizo referencia en su discurso a textos papales, como la encíclica Lumen gentium (1964) que Pablo VI escribió en el revolucionario Concilio Vaticano II, y señaló que “la Iglesia necesita de María para redescubrir su propio rostro femenino, para asemejarse más a ella que, como mujer, Virgen y Madre, representa su modelo y su figura perfecta; para dar espacio a las mujeres y para ser generativa a través de una p

#### URL del artículo

In [6]:
articulo_url = articulo_raw['no_Pagina_url']
# articulo_url = articulo_raw['Pagina']

articulo_url

'https://elpais.com/sociedad/2024-01-01/el-papa-denuncia-la-violencia-contras-las-mujeres-y-reclama-mas-espacio-para-ellas-en-la-iglesia.html'

#### Objeto Article de newspaper

In [7]:
from newspaper import Article

# Crear el objeto Article
# Puedes añadir language='es' si ayuda al parser, aunque suele detectarlo solo.
articulo = Article(articulo_url, language='es')

try:
    # 2. Descargar el HTML
    articulo.download()
    
    # 3. Parsear (analizar) el contenido
    articulo.parse()

except Exception as e:
    print(f"Error al procesar la URL: {e}")

---
# 02/2026 - Variables Generales

---

## 1. ID noticia

In [7]:
def obtener_id_noticia(articulo_raw):
    """
    Extrae y devuelve el IdNoticia de un registro dado.
    """
    articulo_id = articulo_raw['IdNoticia']
    return articulo_id

In [8]:
# --- Uso ---
articulo_id = obtener_id_noticia(articulo_raw)
print(f"{articulo_id=}")

articulo_id='1'


## 2. Medio

In [9]:
from urllib.parse import urlparse

# Función para clasificar el medio según la URL
def clasificar_var_medio(articulo):
    # Extraer el dominio de la URL
    url = articulo.url
    domain = urlparse(url).netloc.lower()
    
    # Diccionario de dominios y categorías
    media_map = {
        "elmundo.es": 1,
        "elpais.com": 2,
        "eldiario.es": 3,
        "20minutos.es": 4,
        "articulo14.com": 5,
        "infolibre.es": 6,
        "lavanguardia.com": 7
    }
    
    # Buscar el dominio en el mapa
    for key, value in media_map.items():
        if key in domain:
            return key, value
    
    # Si no coincide con ninguno, retornar None o 0
    return 0

In [10]:
# --- Uso ---
Medio = clasificar_var_medio(articulo)[0]
Medio_num = clasificar_var_medio(articulo)[1]

print(f"Medio: {Medio}")
print(f"Categoría: {Medio_num}")

Medio: elpais.com
Categoría: 2


## 3. Fecha

In [ ]:
from typing import Optional
from newspaper import Article

def clasificar_var_fecha(articulo: Article) -> Optional[str]:
    """
    Recibe un objeto Article.
    Devuelve la fecha como texto en formato 'dd/mm/aaaa' o None.
    """
    # Verificamos si existe la fecha
    if articulo.publish_date:
        # %d = día (01-31)
        # %m = mes (01-12)
        # %Y = año (cuatro dígitos, ej: 2026)
        return articulo.publish_date.strftime("%d/%m/%Y")
    
    return None

In [12]:
# --- Uso ---
Fecha = clasificar_var_fecha(articulo)
print(f"Fecha: {Fecha}")

Fecha: 01/01/24


## 3a. Mes

In [13]:
def clasificar_var_mes(articulo: Article) -> Optional[int]:
    """
    Recibe un objeto Article procesado.
    Devuelve el mes (MM) como entero o None si no tiene fecha.
    """
    if articulo.publish_date:
        return articulo.publish_date.month
    
    return None

In [14]:
# --- Uso ---
mes = clasificar_var_mes(articulo)
print(f"mes: {mes}")

mes: 1


## 4. Año

In [15]:
def clasificar_var_año(articulo: Article) -> Optional[int]:
    """
    Recibe un objeto Article procesado.
    Devuelve el año (YYYY) como entero o None si no tiene fecha.
    """
    if articulo.publish_date:
        return articulo.publish_date.year
    
    return None

In [16]:
# --- Uso ---
año = clasificar_var_año(articulo)
print(f"año: {año}")

año: 2024


## 5. Número de caracteres

In [17]:
def clasificar_var_caracteres(articulo: Article) -> int:
    """
    Devuelve la cantidad de caracteres del cuerpo del artículo.
    Si no hay texto, devuelve 0.
    """
    if articulo.text:
        # Retorna la longitud del texto
        return len(articulo.text)
    
    return 0

In [18]:
# --- Uso ---
Caracteres = clasificar_var_caracteres(articulo)
print(f"Caracteres: {Caracteres}")

Caracteres: 4697


## 6. Titular. Copia el titular

In [19]:
def clasificar_var_titular(articulo: Article) -> Optional[str]:
    """
    Recibe un objeto Article procesado.
    Devuelve el titular en minúsculas o None si no existe.
    """
    if articulo.title:
        # .strip() quita espacios extra y .lower() pasa a minúsculas
        return articulo.title.strip().lower()
    
    return None

In [20]:
# --- Uso ---
Titular = clasificar_var_titular(articulo)
print(f"Titular: {Titular}")

Titular: el papa denuncia la violencia contra las mujeres y reclama más “espacio” para ellas en la iglesia


## 7a. Nombre Propio Titular

In [39]:
from typing import List
import json

class NombresDetectados(BaseModel):
    # Lista de cadenas con los nombres extraídos
    nombres: List[str] = Field(default_factory=list, description="Lista de nombres propios detectados")
    # Lista de enteros con los códigos correspondientes
    valores: List[int] = Field(default_factory=list, description="Lista de valores clasificados según la tabla")

def clasificar_var_nombre_propio_titular_list(titulo: str) -> NombresDetectados:
    """
    Extrae nombres propios del titular y los clasifica según su género o tipo.
    Devuelve dos listas sincronizadas: nombres y valores.
    """
    
    # Si el título es muy corto, devolvemos listas vacías
    if not titulo or len(titulo) < 3:
        return NombresDetectados(nombres=[], valores=[])

    prompt = f"""
    Analiza el siguiente TITULAR y extrae los nombres propios, entidades, lugares o tecnologías.
    Asigna a cada uno su código numérico correspondiente.

    TITULAR: "{titulo}"

    TABLA DE CÓDIGOS:
    1  = Hombre (Ej: "Pedro Sánchez", "El Papa", "Carlos")
    2  = Mujer (Ej: "María", "Isabel Díaz Ayuso", "Alessandra")
    3  = Grupo Mixto (Ej: "La pareja", "Los vecinos", "Padres")
    4  = Institución, Organización, Empresa, Partido Político (ej: "Google", "PSOE", "La ONU")
    41 = Lugares: Países, Regiones, Ciudades (ej: "España", "Madrid", "Europa")
    42 = Tecnología: Apps, Modelos de IA, Robots, Software (ej: "ChatGPT", "Gemini", "Sora", "TikTok")

    INSTRUCCIONES:
    - Ignora sustantivos comunes que no sean entidades (ej: no extraigas "la policía" si es genérico, pero sí "Mossos d'Esquadra").
    - Distingue bien entre la EMPRESA (OpenAI -> 4) y el PRODUCTO (ChatGPT -> 42).
    - Si no hay nombres propios, devuelve listas vacías.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON con esta estructura exacta:
    {{
        "nombres": ["Nombre1", "Nombre2"],
        "valores": [Codigo1, Codigo2]
    }}
    """

    # --- LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- PARSEO Y VALIDACIÓN ---
    try:
        # Limpieza básica para encontrar el JSON
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1:
            return NombresDetectados(nombres=[], valores=[])
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)
        
        # Validamos con Pydantic
        resultado = NombresDetectados(**data)
        
        # Validación de seguridad: Las listas deben tener el mismo tamaño
        if len(resultado.nombres) != len(resultado.valores):
            # Si hay desajuste, cortamos a la longitud del más corto
            min_len = min(len(resultado.nombres), len(resultado.valores))
            resultado.nombres = resultado.nombres[:min_len]
            resultado.valores = resultado.valores[:min_len]
            
        return resultado

    except (json.JSONDecodeError, ValidationError):
        return NombresDetectados(nombres=[], valores=[])


In [22]:
nombre_propio_titular_list = clasificar_var_nombre_propio_titular_list(articulo.title)
print(f"Nombre Propio Titular Lista Nombres: {nombre_propio_titular_list.nombres}")
print(f"Nombre Propio Titular Lista Valores: {nombre_propio_titular_list.valores}")

Nombre Propio Titular Lista Nombres: ['Papa']
Nombre Propio Titular Lista Valores: [1]


## 7b. Género Nombre Propio Titular

In [40]:
def clasificar_var_nombre_propio_titular(valores: list[int]) -> int:
    """
    Calcula el valor único de protagonismo basado en la lista de entidades detectadas.
    Prioridad absoluta a las personas sobre entidades/lugares.
    """
    
    if not valores:
        return 0

    # 1. CONTADORES DE PERSONAS
    cnt_hombres = valores.count(1)
    cnt_mujeres = valores.count(2)
    
    # Contamos también si el LLM ya detectó grupos mixtos explícitos
    cnt_mixtos_neutro = valores.count(3)
    cnt_mixtos_hombres = valores.count(32)
    cnt_mixtos_mujeres = valores.count(33)

    # Suma total de indicadores humanos
    total_humanos = cnt_hombres + cnt_mujeres + cnt_mixtos_neutro + cnt_mixtos_hombres + cnt_mixtos_mujeres

    # --- FASE 1: FILTRO HUMANO (Prioridad Absoluta) ---
    if total_humanos > 0:
        
        # CASO A: Solo Hombres (Sin mujeres ni grupos mixtos)
        if cnt_hombres > 0 and cnt_mujeres == 0 and cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0:
            return 1
            
        # CASO B: Solo Mujeres (Sin hombres ni grupos mixtos)
        if cnt_mujeres > 0 and cnt_hombres == 0 and cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0:
            return 2

        # CASO C: Mixto (Hay presencia de ambos o indicadores de grupo)
        # Aquí decidimos si es 3, 32 o 33 contando individuos
        
        if cnt_hombres > cnt_mujeres:
            return 32  # Mixto mayoritariamente masculino
        elif cnt_mujeres > cnt_hombres:
            return 33  # Mixto mayoritariamente femenino
        else:
            # Empate técnico (ej: 1 hombre y 1 mujer) o solo había un [3] genérico
            # Si el empate viene de counts explícitos (1 vs 1), es 3.
            # Si viene de grupos (ej: un [32] detectado), respetamos ese matiz.
            if cnt_mixtos_hombres > cnt_mixtos_mujeres:
                return 32
            elif cnt_mixtos_mujeres > cnt_mixtos_hombres:
                return 33
            else:
                return 3 # Empate total o mixto neutro

    # --- FASE 2: NO HUMANOS (Solo si total_humanos == 0) ---
    # Establecemos jerarquía de interés: IA > Entidad > Lugar
    
    if 42 in valores:
        return 42 # Prioridad a Tecnología/IA
    
    if 4 in valores:
        return 4  # Empresas / Instituciones
        
    if 41 in valores:
        return 41 # Lugares (es lo menos informativo)

    # Si todo falla (ej: llegó un código desconocido)
    return 0

In [41]:
nombre_propio_titular = clasificar_var_nombre_propio_titular(nombre_propio_titular_list.valores)
print("nombre_propio_titular", nombre_propio_titular)

nombre_propio_titular 1


In [22]:
import requests
from flair.models import SequenceTagger
from flair.data import Sentence

# 1. Cargar el modelo (se hace una sola vez)
tagger = SequenceTagger.load("flair/ner-spanish-large")

# Caché para no repetir llamadas a la API y ahorrar créditos
cache_generos = {}

def get_gender_cached(name):
    """Consulta Genderize y guarda el resultado en memoria."""
    name = name.lower().capitalize()
    if name in cache_generos:
        return cache_generos[name]
    try:
        # Usamos timeout para que no se quede colgado si falla la red
        response = requests.get(f"https://api.genderize.io?name={name}", timeout=5)
        if response.status_code == 200:
            gen = response.json().get("gender")
            cache_generos[name] = gen
            return gen
    except:
        return None
    return None

def clasificar_var_nombre_propio_titular(titular: str):
    """
    Detecta nombres y devuelve (categoría, lista_nombres).
    Categorías: 1=No hay, 2=Hombre, 3=Mujer, 4=Ambos, 5=Neutro (entidades)
    """
    if not titular:
        return 1, []

    sentence = Sentence(titular)
    tagger.predict(sentence)

    nombres_personas = []
    otras_entidades = []

    for entity in sentence.get_spans('ner'):
        if entity.tag == "PER":
            nombres_personas.append(entity.text)
        else:
            otras_entidades.append(entity.text)

    # Lógica de categorías
    if not nombres_personas and otras_entidades:
        return 5, []
    
    if not nombres_personas:
        return 1, []

    # Determinamos género para la categoría
    hombres, mujeres = 0, 0
    for np in nombres_personas:
        primer_nombre = np.split()[0]
        gen = get_gender_cached(primer_nombre)
        if gen == "male": hombres += 1
        elif gen == "female": mujeres += 1

    if hombres == 0 and mujeres == 0: cat = 1
    elif hombres > mujeres: cat = 2
    elif mujeres > hombres: cat = 3
    else: cat = 4 # Empate o presencia de ambos

    return cat, nombres_personas

def clasificar_var_genero_nombre_propio_titular(lista_nombres):
    """
    Recibe la lista de nombres y devuelve sus géneros.
    Categorías: 1=No hay, 2=Hombre, 3=Mujer, 4=Ambos, 5=Neutro (entidades)
    """
    generos = []
    for nombre_completo in lista_nombres:
        primer_nombre = nombre_completo.split()[0]
        genero = get_gender_cached(primer_nombre)
        # Traducimos a algo más legible
        if genero == "male": generos.append("2")
        elif genero == "female": generos.append("3")
        else: generos.append("desconocido")
    return generos


/home/jggomez/Desktop/IRIS/iris-uc3m/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-02-14 20:40:16.241372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771098016.257255 3425484 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771098016.262184 3425484 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-02-14 20:40:16.278434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perfor

2026-02-14 20:40:30,215 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [23]:
# --- Ejemplo de uso ---
# Titular = "Isabel Díaz Ayuso se reúne con Pedro Sánchez en Madrid"

# # 1. Obtenemos categoría y nombres
# nombre_propio_titular, nombre_propio_titular_lista = clasificar_var_nombre_propio_titular(Titular)

# # 2. Obtenemos los géneros basándonos en esa lista
# genero_nombre_propio_titular = clasificar_var_genero_nombre_propio_titular(nombre_propio_titular_lista)

# print(f"nombre_propio_titular: {nombre_propio_titular}")
# print(f"nombres_encontrados: {nombre_propio_titular_lista}")
# print(f"genero_nombre_propio_titular: {genero_nombre_propio_titular}")


# 1. Obtenemos categoría y nombres
nombre_propio_titular, nombre_propio_titular_lista = clasificar_var_nombre_propio_titular(Titular)

# 2. Obtenemos los géneros basándonos en esa lista
genero_nombre_propio_titular = clasificar_var_genero_nombre_propio_titular(nombre_propio_titular_lista)

print(f"nombre_propio_titular: {nombre_propio_titular}")
print(f"nombres_encontrados: {nombre_propio_titular_lista}")
print(f"genero_nombre_propio_titular: {genero_nombre_propio_titular}")

nombre_propio_titular: 1
nombres_encontrados: []
genero_nombre_propio_titular: []


## 8. Cita en el titular

In [28]:
class CitaTitularValidada(BaseModel):
    # La parte del texto que corresponde a la cita
    cita: str = Field(..., description="El fragmento exacto de la declaración. Si es 1 (No), dejar vacío o poner 'N/A'.")
    
    # El código de clasificación
    tipo: int = Field(..., description="1=No, 2=Directa, 3=Indirecta")

def clasificar_var_cita_titular(titulo: str) -> CitaTitularValidada:
    """
    Analiza si el titular contiene una cita o declaración de alguien.
    Distingue entre directa (textual) e indirecta (parafraseada).
    """
    
    if not titulo:
        return CitaTitularValidada(cita="N/A", tipo=1)

    prompt = f"""
    Analiza el siguiente TITULAR de noticia y detecta si contiene una CITA o DECLARACIÓN de una persona o entidad.

    TITULAR: "{titulo}"

    CLASIFICACIÓN:
    1 = No hay cita. Es un hecho, una descripción del periodista o usa comillas solo para resaltar una palabra (ej: El "caso Koldo").
    2 = Cita Directa. Reproduce palabras textuales. 
        - Pistas: Usa comillas para una frase completa ("Me voy", dijo X) o dos puntos (Sánchez: No dimitiré).
    3 = Cita Indirecta. Parafrasea lo que alguien dijo sin usar comillas para la frase entera.
        - Pistas: Usa verbos de habla (asegura que, dice que, pide, advierte, niega) seguidos de la idea.

    INSTRUCCIONES:
    - En el campo "cita", extrae SOLO el contenido de lo que se ha dicho. Si es tipo 1, pon "N/A".
    - Sé estricto: "Madrid aprueba la ley" es 1 (hecho). "Ayuso dice que Madrid aprobará la ley" es 3 (indirecta).

    Responde SOLO con este JSON:
    {{
        "cita": "texto de la declaración",
        "tipo": numero
    }}
    """

    # Llamada al LLM (asumiendo que tienes tu función consultar_ollama)
    respuesta = consultar_ollama(prompt)
    
    try:
        # Limpieza básica para extraer JSON
        json_str = respuesta[respuesta.find('{'):respuesta.rfind('}')+1]
        data = json.loads(json_str)
        
        return CitaTitularValidada(**data)
        
    except Exception:
        # Fallback en caso de error
        return CitaTitularValidada(cita="Error al procesar", tipo=1)

In [29]:
cita_titular = clasificar_var_cita_titular(articulo.title)
print("Cita Titular Cita: ", cita_titular.cita)
print("Cita Titular Tipo: ", cita_titular.tipo)

Cita Titular Cita:  El Papa denuncia la violencia contra las mujeres y reclama más "espacio" para ellas en la Iglesia
Cita Titular Tipo:  2


## 9a. Protagonistas que aparecen en la información

In [ ]:
class ProtagonistasDetectados(BaseModel):
    # Lista de cadenas con los nombres extraídos
    nombres: List[str] = Field(default_factory=list, description="Lista de nombres únicos detectados en la noticia")
    # Lista de enteros con los códigos correspondientes
    valores: List[int] = Field(default_factory=list, description="Lista de valores clasificados según la tabla")

def clasificar_var_cla_genero_prota_list(texto_noticia: str) -> ProtagonistasDetectados:
    """
    Analiza el cuerpo de la noticia para extraer los protagonistas principales.
    Devuelve listas sincronizadas de nombres únicos y sus códigos.
    """
    
    # Validación básica
    if not texto_noticia or len(texto_noticia) < 10:
        return ProtagonistasDetectados(nombres=[], valores=[])

    # --- PROMPT ---
    prompt = f"""
    Analiza el siguiente TEXTO DE NOTICIA y extrae los protagonistas (personas, entidades, lugares clave).
    
    TEXTO (Fragmento):
    "{texto_noticia}..."

    TABLA DE CÓDIGOS ESTRICTA:
    1  = Hombre (Ej: "Pedro Sánchez", "El Papa", "Carlos")
    2  = Mujer (Ej: "María", "Isabel Díaz Ayuso", "Alessandra")
    3  = Grupo Mixto (Ej: "La pareja", "Los vecinos", "Padres")
    4  = Institución/Empresa (Ej: "Gobierno", "Reuters", "Vatican Media", "PSOE")
    41 = Lugares: Países, Regiones, Ciudades (ej: "España", "Madrid", "Europa")
    42 = Tecnología: Apps, Modelos de IA, Robots, Software (ej: "ChatGPT", "Gemini", "Sora", "TikTok")

    REGLAS OBLIGATORIAS:
    1. Si el nombre es de una sola persona, JAMÁS uses códigos 3, 32 o 33.
    2. Agencias de noticias (Reuters, EFE, Europa Press) son SIEMPRE código 4.
    3. Nombres femeninos (María, Alessandra) son código 2.
    4. Nombres masculinos (Pedro, Francisco) son código 1.
    
    FORMATO JSON EXACTO:
    {{
        "nombres": ["Nombre1", "Nombre2"],
        "valores": [Codigo1, Codigo2]
    }}
    """

    # --- LLAMADA AL MODELO ---
    # Asumiendo que usas tu función 'consultar_ollama'
    # Se recomienda un modelo con buena capacidad de contexto (ej: llama3, mistral, gemma:7b)
    respuesta_texto = consultar_ollama(prompt)

    # --- PARSEO Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1:
            return ProtagonistasDetectados(nombres=[], valores=[])
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)
        
        resultado = ProtagonistasDetectados(**data)
        
        # Sincronización de seguridad
        min_len = min(len(resultado.nombres), len(resultado.valores))
        resultado.nombres = resultado.nombres[:min_len]
        resultado.valores = resultado.valores[:min_len]
            
        return resultado

    except (json.JSONDecodeError, ValidationError):
        return ProtagonistasDetectados(nombres=[], valores=[])

In [37]:
cla_genero_prota_list = clasificar_var_cla_genero_prota_list(articulo.text)
print(f"Nombre Propio Titular Lista Nombres: {cla_genero_prota_list.nombres}")
print(f"Nombre Propio Titular Lista Valores: {cla_genero_prota_list.valores}")

Nombre Propio Titular Lista Nombres: ['Papa Francisco', 'Alessandra Smerilli', 'María', 'Francesco Cassilli', 'Reuters', 'Vatican Media', 'Italia', 'Francesco Cassilli']
Nombre Propio Titular Lista Valores: [1, 2, 2, 1, 4, 4, 41, 1]


## 9b. Género Protagonistas que aparecen en la información

In [44]:
def clasificar_var_cla_genero_prota(valores: list[int]) -> int:
    """
    Calcula el valor único de los protagonistas en el CUERPO de la noticia.
    Toma la lista de códigos detectados y decide el código dominante.
    """
    
    if not valores:
        return 0

    # 1. CONTEO DE CÓDIGOS HUMANOS
    cnt_hombres = valores.count(1)
    cnt_mujeres = valores.count(2)
    
    # Contamos grupos mixtos detectados por el LLM
    cnt_mixtos_neutro = valores.count(3)
    cnt_mixtos_hombres = valores.count(32)
    cnt_mixtos_mujeres = valores.count(33)

    # Suma total de indicadores humanos (Individuales + Grupos)
    total_humanos = cnt_hombres + cnt_mujeres + cnt_mixtos_neutro + cnt_mixtos_hombres + cnt_mixtos_mujeres

    # --- FASE 1: FILTRO HUMANO (Prioridad Absoluta) ---
    if total_humanos > 0:
        
        # CASO A: Solo Hombres (Sin mujeres individuales NI grupos mixtos)
        # Es estricto: si aparece un grupo "33" o "3", ya no es solo hombres.
        if (cnt_hombres > 0 and cnt_mujeres == 0 and 
            cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0):
            return 1
            
        # CASO B: Solo Mujeres (Sin hombres individuales NI grupos mixtos)
        if (cnt_mujeres > 0 and cnt_hombres == 0 and 
            cnt_mixtos_neutro == 0 and cnt_mixtos_hombres == 0 and cnt_mixtos_mujeres == 0):
            return 2

        # CASO C: Mixto (Hay presencia de ambos o hay grupos mixtos)
        # Comparamos cantidades para determinar la mayoría
        
        # C1. Comparación directa de individuos
        if cnt_hombres > cnt_mujeres:
            return 32  # Mixto más hombres
        elif cnt_mujeres > cnt_hombres:
            return 33  # Mixto más mujeres
        
        # C2. Empate de individuos (ej: 0 vs 0, o 2 vs 2). Desempate por grupos.
        else:
            if cnt_mixtos_hombres > cnt_mixtos_mujeres:
                return 32
            elif cnt_mixtos_mujeres > cnt_mixtos_hombres:
                return 33
            else:
                # Empate total (ej: 1 hombre, 1 mujer) o solo grupos neutros (3)
                return 3

    # --- FASE 2: NO HUMANOS (Solo si no hay NINGÚN humano) ---
    
    # Prioridad: Tecnología (42) > Institución (4) > Lugar (41)
    
    if 42 in valores:
        return 42 # IA, Robots, Apps
    
    if 4 in valores:
        return 4  # Empresas, Partidos, Organismos
        
    if 41 in valores:
        return 41 # Países, Ciudades (Fondo)

    # Si llega aquí, es un código desconocido o lista vacía
    return 0

In [45]:
cla_genero_prota = clasificar_var_cla_genero_prota(cla_genero_prota_list.valores)
print(f"Género Protagonistas: {cla_genero_prota}")

Género Protagonistas: 32


### NER (tag de persona)

## 10. Nombre Periodista

In [ ]:
def clasificar_var_nombre_periodista(articulo: articulo) -> str:
    """
    Extrae autores y limpia textos basura como 'Ver Biografía', 'Redacción', etc.
    """
    
    # Lista de palabras/frases que NO queremos en el nombre
    palabras_basura = [
        "ver biografía", "biografía", "ver perfil", "perfil", 
        "ver más", "see profile", "read more", "twitter", 
        "email", "follow", "redacción", "agencia"
    ]

    autores_detectados = []

    # 1. Intentar obtener autores desde el parser
    if articulo.authors:
        for autor in articulo.authors:
            autor_limpio = autor.strip()
            
            # Verificamos si el texto (en minúsculas) contiene alguna palabra basura
            if any(basura in autor_limpio.lower() for basura in palabras_basura):
                # Si es basura pura (ej: "Ver Biografía"), lo ignoramos
                # Pero si es "Lorena Pacho, Ver Biografía", intentamos limpiarlo
                
                # Caso específico que te pasó: eliminar "Ver Biografía" del string
                for basura in palabras_basura:
                    # Reemplazamos la basura por vacío, ignorando mayúsculas/minúsculas es complejo,
                    # así que hacemos un replace simple de las variantes comunes:
                    autor_limpio = autor_limpio.replace("Ver Biografía", "")
                    autor_limpio = autor_limpio.replace("Ver biografía", "")
                
                autor_limpio = autor_limpio.strip(" ,|") # Limpiamos comas o barras sobrantes

            # Si después de limpiar queda algo y no es demasiado corto, lo guardamos
            if len(autor_limpio) > 2:
                autores_detectados.append(autor_limpio)

    # 2. Si encontramos autores limpios, los devolvemos
    if autores_detectados:
        # Eliminamos duplicados usando set() y mantenemos orden
        return ", ".join(list(dict.fromkeys(autores_detectados)))

    # 3. Fallback: Metadatos (si la lista authors falló o se limpió todo)
    meta = articulo.meta_data
    claves_meta = ['author', 'og:author', 'dc.creator', 'byl']
    
    for clave in claves_meta:
        valor = meta.get(clave)
        if valor:
            # A veces los metadatos también traen basura, podrías aplicar limpieza aquí también
            return str(valor).strip()

    # 4. Defecto
    return "Redacción / Otros"

In [ ]:
# --- Uso ---
nombre_periodista = clasificar_var_nombre_periodista(articulo)
print(f"nombre_periodista: {nombre_periodista}")

nombre_periodista: Lorena Pacho


## 11. Género Periodista (Autoría)

In [ ]:
from pydantic import BaseModel, Field, ValidationError
from typing import Optional
import re

# Modelo de validación estricta
class GeneroPeriodistaValidado(BaseModel):
    # Field(...) hace el campo obligatorio
    # ge=0: Greater or equal to 0
    # le=5: Less or equal to 5
    codigo: int = Field(..., ge=0, le=7, description="Código de clasificación de autoría (0-7)")


def clasificar_var_genero_periodista(nombre_periodista: str, nombre_medio:str) -> int:
    """
    Clasifica la autoría considerando el contexto del medio.
    Recibe:
      - nombre_periodista: El texto de la firma (ej: "Redacción", "Juan Pérez")
      - nombre_medio: El nombre del periódico donde se publica (ej: "El País")
    """
    
    # Validación inicial rápida
    if not nombre_periodista or len(nombre_periodista) < 2:
        return 0

    # Prompt enriquecido con el contexto del medio y definiciones
    prompt = f"""
    Contexto:
    Noticia publicada en el medio: "{nombre_medio}".
    Autor/Firma a analizar: "{nombre_periodista}".
    
    Tu misión es clasificar la autoría (0-7) siguiendo estrictamente estas definiciones:

    0 = Ns/Nc: Desconocido, ambiguo o iniciales.
    1 = Hombre: Nombre de persona masculino.
    2 = Mujer: Nombre de persona femenino.
    3 = Mixto: Varios autores de distinto género.
    4 = Otros medios: El autor es otro medio de comunicación (ej: "The New York Times", "Revista Hola").
    5 = Agencia: Agencias de noticias puras (EFE, Europa Press, Reuters, AFP).
    
    6 = Redacción (Periodística): 
        - Firma genérica del propio medio "{nombre_medio}" (ej: "Redacción", "El País", "Editorial").
        - IMPORTANTE: Si el autor es una empresa comercial que NO es un medio de noticias (como Ford, Apple), NO ES 6.
    
    7 = Corporativo (Comercial / Institucional): 
        - Firmado por una empresa comercial, marca de tecnología, coches, banco, etc. (ej: Ford, Meta, Google, BBVA, Zara).
        - Firmado por instituciones gubernamentales o ONGs (ej: Gobierno de España, Greenpeace).
        - Notas de prensa firmadas por la marca.

    INSTRUCCIONES DE PRIORIDAD:
    1. Si "{nombre_periodista}" es una marca conocida (coches, tech, ropa) -> ELIGE 7.
    2. Si "{nombre_periodista}" es igual a "{nombre_medio}" -> ELIGE 6.
    3. Si "{nombre_periodista}" es una Agencia conocida -> ELIGE 5.

    Responde ÚNICAMENTE con el número dígito (0-7).
    """
    # 1. Llamada al modelo (tu función externa)
    respuesta_texto = consultar_ollama(prompt)

    # 2. Extracción del número (Pre-procesamiento)
    # Buscamos el primer dígito que aparezca en el texto
    match = re.search(r'\d+', respuesta_texto)
    
    numero_detectado = int(match.group()) if match else 0

    # 3. Validación con Pydantic
    try:
        # Instanciamos el modelo con el dato detectado
        # Si numero_detectado es 9 (alucinación), Pydantic dará error aquí
        resultado = GeneroPeriodistaValidado(codigo=numero_detectado)
        
        # Si llegamos aquí, es un int válido entre 0 y 5
        return resultado.codigo

    except ValidationError as e:
        print(f"Error de validación Pydantic (Dato inválido: {numero_detectado}): {e}")
        return 0 # Default seguro si la IA alucina un número fuera de rango

In [ ]:
# -- Uso -- 
genero_periodista = clasificar_var_genero_periodista(nombre_periodista=nombre_periodista, nombre_medio=Medio)
print(f"{genero_periodista=}")

genero_periodista=2


## 12. Tema

In [ ]:
import json


class TemaConExplicacion(BaseModel):
    # Validamos que sea un entero entre 0 y 17
    codigo: int = Field(..., ge=0, le=17, description="Código numérico del tema")
    # Añadimos el campo de explicación
    explicacion: str = Field(..., description="Breve justificación de por qué se eligió este tema")

def clasificar_var_tema(titulo: str, texto_cuerpo: str) -> TemaConExplicacion:
    """
    Clasifica el tema y da una explicación.
    Devuelve un objeto Pydantic con .codigo (int) y .explicacion (str).
    """
    
    # 1. Validación rápida
    full_text = f"{titulo} {texto_cuerpo}"
    if not full_text or len(full_text) < 10:
        # Devolvemos objeto vacío/error
        return TemaConExplicacion(codigo=0, explicacion="Texto insuficiente para clasificar.")

    # Recorte para optimizar velocidad
    texto_recortado = texto_cuerpo[:1500]
    
    # 2. Prompt diseñado para JSON
    prompt = f"""
    Analiza la noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea es clasificarla en UNA categoría (1-17) y explicar por qué.
    
    Categorías:
    1 = Científica / Investigación
    2 = Comunicación
    3 = De farándula o espectáculo
    4 = Deportiva
    5 = Economía (Mercados, inflación, consumo)
    6 = Educación/cultura
    7 = Empleo/Trabajo
    8 = Empresa (Corporativo, negocios)
    9 = Judicial
    10 = Medioambiente
    11 = Policial
    12 = Política
    13 = Salud
    14 = Social
    15 = Tecnología
    16 = Transporte
    17 = Otros

    FORMATO DE RESPUESTA OBLIGATORIO (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido con este formato:
    {{
        "codigo": (número entero del 1 al 17),
        "explicacion": "(frase breve justificando tu elección)"
    }}
    """

    # 3. Llamada al modelo
    respuesta_texto = consultar_ollama(prompt)

    # 4. Limpieza y Parsing de JSON
    # A veces los modelos envuelven el JSON en markdown ```json ... ```
    # Buscamos donde empieza '{' y termina '}'
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            raise ValueError("No se encontró JSON en la respuesta")
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str) # Convertimos texto a diccionario Python

        # 5. Validación Pydantic
        resultado = TemaConExplicacion(**data)
        return resultado

    except (json.JSONDecodeError, ValueError, ValidationError) as e:
        print(f"Error parseando respuesta del modelo: {e}")
        # Retorno de seguridad en caso de fallo
        return TemaConExplicacion(codigo=0, explicacion=f"Error técnico: {str(e)}")

In [ ]:
# -- Uso --
tema = clasificar_var_tema(articulo.title, articulo.text)
print(f"ID Tema: {tema.codigo}")
print(f"Explicación: {tema.explicacion}")

ID Tema: 14
Explicación: La noticia trata sobre la denuncia de violencias contra las mujeres, el papel de la mujer en la Iglesia, y la búsqueda de paz y humanidad, lo que lo clasifica claramente como un tema social.


## 12b. Sección

In [ ]:
def clasificar_var_seccion(articulo: Article) -> str:
    """
    Extrae la sección del periódico basándose en metadatos y la URL.
    No usa IA (es más rápido y exacto para este dato estructural).
    """
    
    # --- 1. BUSCAR EN METADATOS (La fuente más fiable) ---
    meta = articulo.meta_data
    
    # Lista de claves comunes donde los medios guardan la sección
    claves_seccion = [
        'section',           # Estándar simple
        'article:section',   # Protocolo Open Graph (Facebook/LinkedIn)
        'og:section',        # Variación Open Graph
        'category',          # WordPress y CMS comunes
        'dc.subject',        # Dublin Core standard
        'ut.section'         # Algunos medios custom
    ]

    for clave in claves_seccion:
        valor = meta.get(clave)
        # A veces el valor es una lista, tomamos el primero
        if isinstance(valor, list):
            valor = valor[0]
        
        if valor and isinstance(valor, str) and len(valor) > 2:
            return valor.strip().title() # Ej: "DEPORTES " -> "Deportes"

    # --- 2. BUSCAR EN LA URL (Si no hay metadatos) ---
    # Ejemplo: https://www.elmundo.es/economia/2024/02/10/noticia.html
    # Queremos extraer "economia"
    
    path = urlparse(articulo.url).path
    segmentos = path.split('/')
    
    # Filtramos segmentos vacíos
    segmentos = [s for s in segmentos if s]

    for segmento in segmentos:
        # Ignoramos segmentos que son años (4 dígitos) o muy cortos (idiomas 'es', 'en')
        if re.match(r'^\d{4}$', segmento): # Es un año (2024)
            continue
        if re.match(r'^\d{1,2}$', segmento): # Es un día o mes (10, 02)
            continue
        if len(segmento) <= 2: # Es un código de idioma (es, en, cat)
            continue
        if segmento in ['noticia', 'articulo', 'story', 'news']: # Palabras genéricas
            continue
            
        # Si pasa los filtros, asumimos que es la sección
        # Reemplazamos guiones por espacios (ej: "ciencia-y-salud" -> "Ciencia Y Salud")
        return segmento.replace('-', ' ').title()

    # --- 3. DEFECTO ---
    return "General"

In [ ]:
# -- Uso --
seccion = clasificar_var_seccion(articulo)
print(f"Sección detectada: {seccion}")

Sección detectada: Sociedad


---
# 02/2026 - Variables específicas IRIS

---

## 13. IA tema central

In [ ]:
class IaTemaCentralConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No es tema central, 2=Sí es tema central")
    # Campo nuevo
    explicacion: str = Field(..., description="Justificación de la jerarquía de la información")

def clasificar_var_ia_tema_central(titulo: str, texto_cuerpo: str) -> IaTemaCentralConExplicacion:
    """
    Determina si la Inteligencia Artificial es el TEMA CENTRAL de la noticia.
    Devuelve objeto con .codigo y .explicacion.
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    palabras_clave_ia = [
        "inteligencia artificial", "artificial intelligence", "ia ", "ai ", 
        "chatgpt", "gpt", "llm", "machine learning", "aprendizaje automático",
        "red neuronal", "deep learning", "midjourney", "dall-e", "bard", "gemini",
        "copilot", "algoritmo generativo", "sam altman", "openai", "nvidia"
    ]
    
    # Si ninguna palabra clave está presente, asumimos directamente que NO (1)
    if not any(palabra in texto_completo for palabra in palabras_clave_ia):
        return IaTemaCentralConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos relacionados con la Inteligencia Artificial."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    # Recortamos texto para centrar la atención en el inicio (donde suele estar el tema central)
    texto_recortado = texto_cuerpo[:2500]
    
    prompt = f"""
    Analiza la jerarquía de la información en esta noticia:
    
    TÍTULO: "{titulo}"
    TEXTO: "{texto_recortado}..."

    Objetivo: Determinar si la Inteligencia Artificial (IA) es el TEMA PRINCIPAL y PROTAGONISTA.
    
    Criterios de clasificación:

    1 = No (Mención secundaria / Otro tema):
        - La IA se menciona al final o de pasada.
        - Es un discurso (Papa, Políticos) sobre varios temas y la IA es solo uno más.
        - La IA es una herramienta secundaria (ej: "Policía usa IA para un robo", el tema es el robo).
        - El Título NO menciona tecnología o IA.

    2 = Sí (Tema Central):
        - La noticia gira completamente en torno a la IA (avances, regulación, peligros, inversiones).
        - La IA es el sujeto principal del Título.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Breve justificación analizando si la IA es protagonista o secundaria)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        # Buscamos el bloque JSON
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return IaTemaCentralConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación Pydantic
        return IaTemaCentralConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        # Fallback seguro
        return IaTemaCentralConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
ia_tema_central = clasificar_var_ia_tema_central(articulo.title, articulo.text)

print(f"Código: {ia_tema_central.codigo}")
print(f"Razón: {ia_tema_central.explicacion}")

Código: 1
Razón: La noticia se centra en el discurso del Papa Francisco sobre la violencia contra las mujeres, el papel de la mujer en la Iglesia y la necesidad de 'espacio' para ellas. La IA se menciona brevemente al final, en relación a la asamblea del sínodo, pero no es el tema principal ni el protagonista de la noticia.


## 14. Explicación sobre el significado de la IA

In [ ]:
class IaSignificadoConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No explica significado, 2=Sí explica significado")
    # Campo nuevo
    explicacion: str = Field(..., description="Justificación: ¿Hay definiciones técnicas o es solo mención?")

def clasificar_var_significado_ia(titulo: str, texto_cuerpo: str) -> IaSignificadoConExplicacion:
    """
    Detecta si el artículo explica o define QUÉ ES la IA o CÓMO FUNCIONA.
    Devuelve un objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Si no hay palabras "inteligencia" o "algoritmo", difícilmente explicará qué son.
    keywords_tecnicas = ["inteligencia", "ia ", "ai ", "algoritmo", "red neuronal", "modelo de lenguaje"]
    
    if not any(k in texto_completo for k in keywords_tecnicas):
        return IaSignificadoConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos técnicos básicos para ofrecer una definición de IA."
        )

    # --- 2. PROMPT EN FORMATO JSON ---
    # Recortamos el texto (buscamos definiciones, suelen estar al principio)
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza el siguiente texto periodístico con enfoque pedagógico:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si el artículo contiene una EXPLICACIÓN o DEFINICIÓN sobre qué es la Inteligencia Artificial (IA) o cómo funciona técnicamente.

    Criterios de clasificación:
    
    1 = No (Mera mención o uso):
        - El artículo habla de herramientas (ChatGPT, Bard) o noticias de empresas sin explicar qué son.
        - Habla de "la IA" como un sujeto abstracto ("la IA cambiará el mundo") sin definirla.
        - Ej: "Google lanzó su nueva IA ayer". Aquí NO se aprende qué es la tecnología.

    2 = Sí (Didáctico / Definitorio):
        - El texto tiene intención educativa.
        - Contiene frases tipo: "La IA generativa funciona prediciendo el siguiente token...", "Los LLM son modelos entrenados con...".
        - Explica la diferencia técnica entre tipos de IA.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Breve frase justificando si hay definiciones técnicas o solo menciones)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        # Buscamos el bloque JSON por si el modelo añade texto extra
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return IaSignificadoConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación con Pydantic
        return IaSignificadoConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return IaSignificadoConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
significado_ia = clasificar_var_significado_ia(articulo.title, articulo.text)

print(f"Código: {significado_ia.codigo}")
print(f"Razón: {significado_ia.explicacion}")

Código: 1
Razón: El artículo menciona el uso de la IA a través de referencias al Papa Francisco y sus discursos, pero no ofrece ninguna definición técnica o explicación de cómo funciona la IA.


## 15. Se menciona la IA en algún momento

In [ ]:
class MencionIaConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No menciona IA, 2=Sí menciona IA")
    # Explicación generada automáticamente por Python
    explicacion: str = Field(..., description="Justificación exacta (qué palabra o sigla se encontró)")


def clasificar_var_menciona_ia(titulo: str, texto_cuerpo: str) -> MencionIaConExplicacion:
    """
    Detecta si se menciona la IA y lista TODAS las palabras clave encontradas.
    Usa Regex y palabras clave (No requiere Ollama).
    """
    
    # Unificamos texto
    texto_completo = f"{titulo} \n {texto_cuerpo}"
    texto_lower = texto_completo.lower()
    
    # Creamos un conjunto (set) para evitar palabras repetidas
    palabras_encontradas = set()
    
    # --- 1. BÚSQUEDA DE CONCEPTOS CLAROS ---
    palabras_clave = [
        "inteligencia artificial", "artificial intelligence",
        "machine learning", "aprendizaje automático", "deep learning",
        "redes neuronales", "chatgpt", "generative ai", "ia generativa",
        "openai", "midjourney", "dall-e", "bard", "gemini", "copilot",
        "large language model", " llm ", "algoritmos generativos",
        "sam altman", "sora", "claude 3", "llama 3", "mistral",
        "vision pro", "neuralink"
    ]
    
    # Iteramos sobre todas las palabras y si están, las añadimos al set
    for frase in palabras_clave:
        if frase in texto_lower:
            palabras_encontradas.add(frase.strip()) # strip para quitar espacios de " llm "

    # --- 2. BÚSQUEDA DE SIGLAS "IA" o "AI" (Case Sensitive) ---
    # Usamos re.findall para encontrar TODAS las ocurrencias, no solo la primera
    patron_siglas = r'\b(IA|AI|I\.A\.|A\.I\.)\b'
    coincidencias_siglas = re.findall(patron_siglas, texto_completo)
    
    # Añadimos las siglas encontradas al conjunto
    for sigla in coincidencias_siglas:
        palabras_encontradas.add(sigla)

    # --- 3. CONSTRUCCIÓN DE RESPUESTA ---
    
    # Si el conjunto tiene elementos, es un SÍ (2)
    if palabras_encontradas:
        # Convertimos el set a una lista ordenada y la unimos con comas
        lista_final = ", ".join(sorted(palabras_encontradas))
        return MencionIaConExplicacion(
            codigo=2,
            explicacion=lista_final
        )

    # Si el conjunto está vacío, es un NO (1)
    return MencionIaConExplicacion(
        codigo=1,
        explicacion="No se encontraron términos, siglas ni conceptos relacionados con la Inteligencia Artificial en el texto."
    )

In [ ]:
# -- Uso --
menciona_ia = clasificar_var_menciona_ia(articulo.title, articulo.text)

print(f"Código: {menciona_ia.codigo}")
print(f"Razón: {menciona_ia.explicacion}")

Código: 2
Razón: inteligencia artificial


## 16. Referencias en la noticia a políticas en materia de género e igualdad

In [ ]:
class ReferenciaPoliticasGeneroConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No referencia políticas, 2=Sí referencia políticas de género")
    # Campo nuevo para el razonamiento
    explicacion: str = Field(..., description="Justificación de la decisión")


def clasificar_var_referencia_politicas_genero(titulo: str, texto_cuerpo: str) -> ReferenciaPoliticasGeneroConExplicacion:
    """
    Detecta si la noticia hace referencia a POLÍTICAS, LEYES o DEBATES sobre 
    género, igualdad, feminismo o violencia machista.
    Devuelve objeto con .codigo y .explicacion.
    """
    
    # Unificamos y limpiamos texto para el filtro
    texto_completo = (titulo + " " + texto_cuerpo).lower()
    
    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    raices_clave = [
        "igualdad", "género", "genero", "femin", "mujer", "machis", 
        "brecha", "paridad", "sexis", "patriarca", "trans ", "lgtbi",
        "conciliación", "techo de cristal", "violencia vicaria", "víctima",
        "ley", "ministerio", "protesta", "derechos" # Añadido contexto político/legal
    ]
    
    # Verificamos si hay al menos una palabra de género Y contexto político/social
    # (Simplificado: si no hay ninguna raíz clave, descartamos).
    if not any(raiz in texto_completo for raiz in raices_clave):
        return ReferenciaPoliticasGeneroConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos clave relacionados con género, igualdad o políticas sociales."
        )

    # --- 2. PROMPT ESPECÍFICO (Modo JSON) ---
    # Recortamos el texto para no saturar el contexto
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la siguiente noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si el texto hace referencia a POLÍTICAS DE GÉNERO, LEYES DE IGUALDAD o DEBATES SOBRE DERECHOS DE LA MUJER.

    Criterios de clasificación:
    1 = No:
        - La mujer es mencionada solo como protagonista de un hecho (ej: "La alcaldesa inauguró la feria").
        - Sucesos o crímenes sin contexto social/legal.
    
    2 = Sí:
        - Menciona leyes, cuotas, paridad o medidas gubernamentales sobre igualdad.
        - Habla de violencia machista/género como problema estructural o legal.
        - Trata sobre feminismo, 8M, brecha salarial o discriminación laboral.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Frase breve justificando si se trata de política/derechos o es solo una mención circunstancial)"
    }}
    """

    # --- 3. Llamada al modelo ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. Procesamiento JSON ---
    try:
        # Buscamos el JSON dentro de la respuesta (por si el modelo añade texto extra)
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            # Fallback si no hay JSON
            return ReferenciaPoliticasGeneroConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación final con Pydantic
        return ReferenciaPoliticasGeneroConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return ReferenciaPoliticasGeneroConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
referencia_politicas_genero = clasificar_var_referencia_politicas_genero(articulo.title, articulo.text)

print(f"Código: {referencia_politicas_genero.codigo}")
print(f"Razón: {referencia_politicas_genero.explicacion}")

Código: 2
Razón: El texto aborda la violencia contra las mujeres como un problema estructural, refiriéndose a la necesidad de 'espacio' para las mujeres en la Iglesia y a la importancia de 'cuidarlas, valorarlas' y 'ser generativa a través de una pastoral hecha de cuidado y solicitud'. Además, menciona la influencia de documentos como la encíclica Lumen Gentium, vinculada al Concilio Vaticano II, y el papel de María como modelo, directamente relacionado con la defensa de los derechos de la mujer.


## 17. Denuncia a la desigualdad de género

In [ ]:
class DenunciaDesigualdadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No denuncia, 2=Sí denuncia desigualdad")
    # Nueva explicación
    explicacion: str = Field(..., description="Justificación de por qué se considera denuncia o no")


def clasificar_var_denuncia_desigualdad_genero(titulo: str, texto_cuerpo: str) -> DenunciaDesigualdadConExplicacion:
    """
    Detecta si la noticia DENUNCIA desigualdad y explica por qué.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()
    
    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    palabras_activadoras = [
        "desigualdad", "discriminaci", "brecha", "violencia", "machis", 
        "patriarca", "acos", "abus", "víctima", "feminici", "sexismo",
        "techo de cristal", "precariedad", "injusticia", "derechos de las mujeres",
        "igualdad real", "conciliación", "paridad"
    ]
    
    # Si NO hay palabras clave, retornamos 1 directamente con explicación automática
    if not any(p in texto_completo for p in palabras_activadoras):
        return DenunciaDesigualdadConExplicacion(
            codigo=1, 
            explicacion="El texto no contiene términos relacionados con género, desigualdad o violencia machista."
        )

    # --- 2. PROMPT PARA JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza el enfoque de esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea es determinar si el texto DENUNCIA o VISIBILIZA un problema de desigualdad de género.

    Criterios:
    1 = No (Neutro/Sucesos): Solo narra hechos sin crítica social, o habla de mujeres exitosas sin mencionar dificultades de género.
    2 = Sí (Denuncia/Crítica): Critica el machismo, aporta datos de brechas, denuncia violencia sistémica o cubre protestas feministas.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Frase breve justificando si hay denuncia social o es meramente informativo)"
    }}
    """

    # --- 3. LLAMADA A OLLAMA ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. PROCESAMIENTO DE RESPUESTA ---
    try:
        # Limpieza de bloques de código markdown si los hay
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            # Fallback si el modelo no devuelve JSON
            return DenunciaDesigualdadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        # Validación Pydantic
        return DenunciaDesigualdadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        # Si algo falla en el parseo, devolvemos un objeto seguro
        return DenunciaDesigualdadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )
    

In [ ]:
# -- Uso --
denuncia_desigualdad_genero = clasificar_var_denuncia_desigualdad_genero(articulo.title, articulo.text)

print(f"Código: {denuncia_desigualdad_genero.codigo}")
print(f"Razón: {denuncia_desigualdad_genero.explicacion}")

Código: 2
Razón: El texto denuncia la violencia contra las mujeres y, específicamente, la necesidad de que la Iglesia dé más 'espacio' a las mujeres, invocando la figura de María como modelo.  Se refiere a la 'violación de Dios' causada por la violencia contra mujeres y al imperativo de que la Iglesia se 'redescubra su rostro femenino', indicando una crítica a las estructuras y prácticas que perpetúan la desigualdad de género.


## 18. Presencia de mujeres racializadas en la noticia

In [ ]:
class MujeresRacializadasConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No aparecen, 2=Sí aparecen mujeres racializadas")
    # Justificación
    explicacion: str = Field(..., description="Detalle sobre quiénes son las mujeres detectadas y su contexto étnico")

def clasificar_var_mujeres_racializadas_noticias(titulo: str, texto_cuerpo: str) -> MujeresRacializadasConExplicacion:
    """
    Detecta la presencia o mención de mujeres racializadas (no blancas/caucásicas) en la noticia.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Buscamos términos que sugieran diversidad étnica, racial o contextos migratorios.
    # Si no aparece NADA de esto, asumimos que se habla de mujeres blancas o el tema no es racial.
    
    terminos_clave = [
        "racializada", "negra", "afro", "etnia", "raza", "indígena", 
        "gitana", "romaní", "latina", "hispana", "asiática", "árabe", 
        "musulmana", "morocc", "marroquí", "subsahariana", "migrante", 
        "refugiada", "islam", "velo", "hijab", "mestiza", "mulata",
        "origen", "nacionalidad", "extranjera", "diversidad"
    ]
    
    # Nota: Este filtro es laxo para no descartar falsos negativos, 
    # pero ayuda a limpiar noticias de política nacional estándar (ej: Ayuso, Montero).
    if not any(t in texto_completo for t in terminos_clave):
        return MujeresRacializadasConExplicacion(
            codigo=1,
            explicacion="El texto no contiene marcadores explícitos de diversidad étnica o racial."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la representación de las personas en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si en la noticia aparecen, se mencionan o protagonizan **MUJERES RACIALIZADAS**.
    
    Definición de "Mujer Racializada" para este análisis:
    Mujeres que son percibidas socialmente como no blancas en un contexto occidental. Incluye:
    - Mujeres negras / afrodescendientes.
    - Mujeres latinas / sudamericanas.
    - Mujeres asiáticas.
    - Mujeres árabes / magrebíes / musulmanas (contexto cultural-étnico).
    - Mujeres indígenas.
    - Mujeres gitanas / romaníes.

    Criterios de clasificación:
    
    1 = No:
        - Solo aparecen mujeres blancas / caucásicas (ej: políticas europeas, actrices de Hollywood blancas).
        - No se menciona el origen étnico y por el contexto se asume hegemonía blanca.
        - Se habla de "inmigrantes" en general sin especificar mujeres.

    2 = Sí:
        - Aparece explícitamente una mujer descrita por su etnia u origen (ej: "la activista afroamericana", "la cantante colombiana").
        - Se menciona a una figura pública conocida por ser racializada (ej: Kamala Harris, Rihanna, Salma Hayek, Zendaya) aunque no se diga su raza explícitamente en el texto.
        - Noticias sobre colectivos específicos (ej: "Las mujeres afganas", "Las temporeras marroquíes").

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica qué mujer o colectivo racializado se ha detectado y por qué)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresRacializadasConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresRacializadasConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresRacializadasConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_racializadas_noticias = clasificar_var_mujeres_racializadas_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_racializadas_noticias.codigo}")
print(f"Razón: {mujeres_racializadas_noticias.explicacion}")

Código: 1
Razón: El texto no contiene marcadores explícitos de diversidad étnica o racial.


## 19. Presencia de mujeres con discapacidad en la noticia

In [ ]:
class MujeresConDiscapacidadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No aparecen, 2=Sí aparecen mujeres con discapacidad")
    # Justificación
    explicacion: str = Field(..., description="Detalle sobre quiénes son las mujeres detectadas y su contexto de discapacidad")

def clasificar_var_mujeres_con_discapacidad_noticias(titulo: str, texto_cuerpo: str) -> MujeresConDiscapacidadConExplicacion:
    """
    Detecta la presencia o mención explícita de mujeres con discapacidad o diversidad funcional.
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Palabras clave que sugieren discapacidad, diversidad funcional o condiciones específicas.
    # Si no aparece ninguna, descartamos la noticia.
    
    terminos_clave = [
        "discapacidad", "diversidad funcional", "silla de ruedas", "movilidad reducida",
        "ciega", "sorda", "sordomuda", "invidente", "autis", " tea ", "asperger",
        "síndrome de down", "parálisis", "cerebral", "amputada", "prótesis",
        "salud mental", "trastorno", "bipolar", "esquizofren", "depresio", # En contextos de discapacidad psicosocial
        "dependencia", "capacitism", "paralímpic", "once", "cermi"
    ]
    
    if not any(t in texto_completo for t in terminos_clave):
        return MujeresConDiscapacidadConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos relacionados con la discapacidad o diversidad funcional."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la representación de las personas en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si en la noticia aparecen, se mencionan o protagonizan **MUJERES CON DISCAPACIDAD**.

    Criterios de clasificación:
    
    1 = No:
        - Se menciona discapacidad, pero en HOMBRES (ej: "El atleta paralímpico ganó el oro").
        - Se usan términos metafóricos (ej: "La justicia es ciega", "parálisis política").
        - Son lesiones temporales (ej: "La jugadora se rompió la pierna y estará baja un mes").
        - Se habla de discapacidad en general (leyes, barreras) sin mencionar a ninguna mujer o colectivo femenino específico.

    2 = Sí:
        - Aparece una mujer (o niña) con discapacidad física, sensorial, intelectual o psicosocial.
        - Se habla de colectivos específicos (ej: "Las mujeres con discapacidad sufren más violencia").
        - Se menciona a deportistas paralímpicas, activistas con diversidad funcional, etc.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica quién es la mujer y cuál es su discapacidad o contexto)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    # Gemma 4b suele ser bueno distinguiendo género en estos contextos
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresConDiscapacidadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresConDiscapacidadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresConDiscapacidadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_con_discapacidad_noticias = clasificar_var_mujeres_con_discapacidad_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_con_discapacidad_noticias.codigo}")
print(f"Razón: {mujeres_con_discapacidad_noticias.explicacion}")

Código: 1
Razón: El texto no contiene términos relacionados con la discapacidad o diversidad funcional.


## 20. Presencia de diversidad generacional en las mujeres que aparecen

In [ ]:
class MujeresGeneracionalidadConExplicacion(BaseModel):
    # 1 = No, 2 = Sí
    codigo: int = Field(..., ge=1, le=2, description="1=No hay diversidad generacional, 2=Sí hay diversidad (niñas, ancianas o mezcla)")
    # Justificación
    explicacion: str = Field(..., description="Detalle de las edades o generaciones identificadas en la noticia")

def clasificar_var_mujeres_generacionalidad_noticias(titulo: str, texto_cuerpo: str) -> MujeresGeneracionalidadConExplicacion:
    """
    Detecta si en la noticia aparecen mujeres de **distintas generaciones** o de 
    **edades no hegemónicas** (niñas, adolescentes o ancianas).
    Devuelve objeto con .codigo (1/2) y .explicacion (str).
    """
    
    # Unificamos texto
    texto_completo = (titulo + " " + texto_cuerpo).lower()

    # --- 1. FILTRO DE EFICIENCIA (Heurística) ---
    # Buscamos marcadores de edad extremos o relaciones intergeneracionales.
    # Si no aparecen, asumimos que son adultos estándar (lo más común en noticias).
    
    terminos_edad = [
        # Infancia / Juventud
        "niña", "adolescente", "joven", "menor", "escolar", "alumna", "estudiante", 
        "chica", "hija", "infantil", "bebé", "generación z",
        # Vejez / Tercera Edad
        "anciana", "abuela", "jubilada", "mayor", "tercera edad", "senior", 
        "vejez", "pensionista", "octogenaria", "nonagenaria", "vieja", "residencia",
        # Relacional
        "madre", "nieta", "familia", "generaciones", "intergeneracional"
    ]
    
    if not any(t in texto_completo for t in terminos_edad):
        return MujeresGeneracionalidadConExplicacion(
            codigo=1,
            explicacion="El texto no contiene términos que sugieran diversidad de edades (niñas, ancianas) o relaciones intergeneracionales."
        )

    # --- 2. PROMPT CON SOLICITUD DE JSON ---
    texto_recortado = texto_cuerpo[:2000]
    
    prompt = f"""
    Analiza la edad y las generaciones de las mujeres en esta noticia:
    Título: "{titulo}"
    Extracto: "{texto_recortado}..."

    Tu tarea: Determinar si hay **DIVERSIDAD GENERACIONAL** en la representación femenina.

    Criterios de clasificación:
    
    1 = No (Representación Estándar):
        - Solo aparecen mujeres adultas en edad laboral típica (aprox 25-60 años). Ej: Políticas, profesionales, empresarias.
        - Se menciona "madre" solo como dato biográfico sin relevancia en la historia (ej: "es madre de dos hijos").
        - No se especifica la edad y se asume adultez.

    2 = Sí (Diversidad / Edades no hegemónicas):
        - Aparecen **Niñas o Adolescentes** con voz propia o como protagonistas.
        - Aparecen **Mujeres Mayores / Ancianas / Jubiladas** (Visibilidad de la tercera edad).
        - Hay un enfoque **Intergeneracional**: Se habla de madres e hijas, abuelas y nietas, o el impacto de un tema en distintas generaciones de mujeres.

    FORMATO DE RESPUESTA (JSON):
    Responde ÚNICAMENTE con un objeto JSON válido:
    {{
        "codigo": (1 o 2),
        "explicacion": "(Indica qué edades o relación generacional se ha detectado)"
    }}
    """

    # --- 3. LLAMADA AL MODELO ---
    # Usamos Gemma 4b (o tu modelo preferido)
    respuesta_texto = consultar_ollama(prompt)

    # --- 4. EXTRACCIÓN Y VALIDACIÓN ---
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1 or fin == 0:
            return MujeresGeneracionalidadConExplicacion(
                codigo=1, 
                explicacion="Error: El modelo no devolvió un formato JSON válido."
            )
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)

        return MujeresGeneracionalidadConExplicacion(**data)

    except (json.JSONDecodeError, ValidationError) as e:
        return MujeresGeneracionalidadConExplicacion(
            codigo=1, 
            explicacion=f"Error técnico al procesar la respuesta: {str(e)}"
        )

In [ ]:
# -- Uso --
mujeres_generacionalidad_noticias = clasificar_var_mujeres_generacionalidad_noticias(articulo.title, articulo.text)

print(f"Código: {mujeres_generacionalidad_noticias.codigo}")
print(f"Razón: {mujeres_generacionalidad_noticias.explicacion}")

Código: 1
Razón: La noticia se centra en el Papa Francisco y en su visión sobre el papel de la mujer en la Iglesia. Si bien se menciona el papel de María y de las madres, no hay ninguna mención explícita a niñas, adolescentes o mujeres mayores. La representación femenina se limita a la figura de la Virgen y a la referencia general a las madres sin detalles sobre sus edades o generaciones.


## 21. Tiene fotografías

In [53]:
from typing import List, Any

class FotografiasValidadas(BaseModel):
    # Código: 1=No, 2=Sí
    codigo: int = Field(..., description="1 = No tiene fotos, 2 = Sí tiene fotos.")
    
    # Cantidad total
    cantidad: int = Field(..., ge=0, description="Número total de fotografías editoriales detectadas.")
    
    # Lista de links (URLs)
    evidencias: List[str] = Field(default_factory=list, description="Lista de URLs de las imágenes encontradas.")

def clasificar_var_fotografias(articulo: Any) -> FotografiasValidadas:
    """
    Analiza las imágenes del artículo (Top Image + Cuerpo).
    Filtra iconos, basura y publicidad.
    
    Args:
        articulo: Objeto 'Article' de la librería newspaper3k ya descargado y parseado.
    """
    
    # Usamos un set para evitar duplicados (ej: si la top_image también sale en el texto)
    imagenes_reales = set()
    
    # --- 1. IMAGEN DE PORTADA (TOP IMAGE) ---
    if articulo.top_image and len(articulo.top_image) > 10:
        # Filtro básico: que sea una URL válida y no vacía
        imagenes_reales.add(articulo.top_image)

    # --- 2. IMÁGENES DEL CUERPO ---
    # Usamos clean_top_node (lxml object) que contiene solo el texto principal limpio
    nodo_texto = articulo.clean_top_node 
    
    if nodo_texto is not None:
        # Buscamos todas las etiquetas <img>
        imgs_en_texto = nodo_texto.xpath('.//img')
        
        for img in imgs_en_texto:
            src = img.get('src')
            if not src:
                continue
            
            # Normalizamos a minúsculas para chequear
            src_lower = src.lower()
            
            # --- FILTROS ANTI-BASURA (Heurística) ---
            
            # A. Descartar formatos que suelen ser de interfaz (iconos, spacers)
            if src_lower.endswith(('.svg', '.gif', '.ico')):
                continue
                
            # B. Palabras prohibidas (indican publicidad, tracking o diseño web)
            palabras_prohibidas = [
                'logo', 'icon', 'avatar', 'profile', 'pixel', 'spacer', 
                'doubleclick', 'adserver', 'banner', 'button', 'social',
                'facebook', 'twitter', 'whatsapp', 'share', 'sprite',
                'author', 'comment'
            ]
            
            if any(palabra in src_lower for palabra in palabras_prohibidas):
                continue

            # C. Descartar por dimensiones diminutas (si el HTML las tiene)
            # Muchos "pixels" de tracking son de 1x1
            width = img.get('width')
            height = img.get('height')
            
            # Si tiene width/height y es menor a 100px, seguramente no es una foto editorial
            if width and width.isdigit() and int(width) < 100:
                continue
            if height and height.isdigit() and int(height) < 100:
                continue

            # Si pasa los filtros, añadimos la URL
            imagenes_reales.add(src)

    # --- 3. CONSTRUCCIÓN DE RESPUESTA ---
    lista_urls = list(imagenes_reales)
    cantidad_final = len(lista_urls)
    
    # Lógica de código: 2 si hay fotos, 1 si no
    codigo_final = 2 if cantidad_final > 0 else 1

    return FotografiasValidadas(
        codigo=codigo_final,
        cantidad=cantidad_final,
        evidencias=lista_urls
    )

In [57]:
# --- Uso ---
tiene_fotografías = clasificar_var_fotografias(articulo).codigo
print(f"tiene_fotografías: {tiene_fotografías}")
print(f"evidencias: {clasificar_var_fotografias(articulo).evidencias}")

tiene_fotografías: 2
evidencias: ['https://imagenes.elpais.com/resizer/v2/GRTTGV7Y4ZKFDVK4U3GUUYUFIQ.jpg?auth=3d17bb971b271d189c1b6328e0e021caa8c8564205be253ff3fd874f05e20526&width=1200', 'https://imagenes.elpais.com/resizer/v2/BIL6F6XXBACFYNHAVX2ZOCOSEQ.jpg?auth=860ae2406676a882f13929e253d0da8a5a8737f37e136ebafe0ab14a6efdc776&width=414', 'https://imagenes.elpais.com/resizer/v2/4ARYAUF2ZAV3NZ3PP3N6XUV6PM.jpg?auth=dc0f4ec4476886d2edaf94a765bf5325c787d36cf24ffbab6d8fd277c1248640&width=414']


## 22. Número de fotografías

In [56]:
# --- Uso ---
numero_fotografias = clasificar_var_fotografias(articulo).cantidad
print(f"cantidad: {numero_fotografias}")

cantidad: 3


## 23. Tiene Fuentes

In [47]:
class FuentesValidadas(BaseModel):
    # Un solo número entero: 2 si hay fuentes, 1 si no hay
    codigo: int = Field(..., description="1 = No tiene fuentes, 2 = Sí tiene fuentes.")
    
    # La lista de evidencias (nombres de las fuentes)
    evidencias: List[str] = Field(default_factory=list, description="Lista de nombres de las fuentes detectadas.")
    
    # Cantidad total
    cantidad: int = Field(..., description="Número total de fuentes.")

def clasificar_var_tiene_fuentes(texto_noticia: str) -> FuentesValidadas:
    """
    Determina si la noticia tiene fuentes.
    Retorna codigo=2 si encuentra al menos una, codigo=1 si no encuentra nada.
    """
    
    # 1. Validación inicial
    if not texto_noticia or len(texto_noticia) < 50:
        return FuentesValidadas(codigo=1, evidencias=[], cantidad=0)

    # 2. Recorte (Analizamos el principio del texto donde se atribuyen las fuentes)
    texto_analisis = texto_noticia[:3500]

    # 3. Prompt: Solo pedimos la lista de nombres
    prompt = f"""
    Analiza el texto y extrae una lista de las FUENTES de información explícitas (personas, entidades, documentos) a las que se atribuyen los datos.

    TEXTO: "{texto_analisis}..."

    INSTRUCCIONES:
    - Busca verbos de atribución: "según X", "dijo Y", "informó Z", "fuentes de...", "el informe de...".
    - Si es un artículo de opinión sin datos externos, la lista debe estar vacía.

    Responde SOLO con este JSON:
    {{
        "fuentes": ["Nombre Fuente 1", "Nombre Fuente 2"]
    }}
    """

    # 4. Llamada al LLM
    respuesta_texto = consultar_ollama(prompt)

    # 5. Lógica Python (Determinista)
    try:
        inicio = respuesta_texto.find('{')
        fin = respuesta_texto.rfind('}') + 1
        
        if inicio == -1:
            return FuentesValidadas(codigo=1, evidencias=[], cantidad=0)
            
        json_str = respuesta_texto[inicio:fin]
        data = json.loads(json_str)
        
        # Extraemos la lista limpia
        lista_fuentes = data.get("fuentes", [])
        cantidad = len(lista_fuentes)
        
        # --- AQUÍ ESTÁ EL CAMBIO ---
        if cantidad > 0:
            # Si hay elementos -> Código 2 (Sí)
            return FuentesValidadas(
                codigo=2,
                evidencias=lista_fuentes,
                cantidad=cantidad
            )
        else:
            # Si la lista está vacía -> Código 1 (No)
            return FuentesValidadas(
                codigo=1,
                evidencias=[],
                cantidad=0
            )

    except Exception:
        return FuentesValidadas(codigo=1, evidencias=[], cantidad=0)

In [50]:
# --- Uso ---
tiene_fuentes = clasificar_var_tiene_fuentes(articulo.text).codigo
print(f"tiene_fuentes: {tiene_fuentes}")
print(f"evidencias: {clasificar_var_tiene_fuentes(articulo.text).evidencias}")


tiene_fuentes: 2
evidencias: ['Papa Francisco', 'Ministerio del Interior italiano', 'REUTERS', 'VATICAN MEDIA', 'Alessandra Smerilli']


## 24. Número de Fuentes

In [51]:
# --- Uso ---
print(f"cantidad: {clasificar_var_tiene_fuentes(articulo.text).cantidad}")

cantidad: 5


---
# 02/2026 - Experimento Interspeech (Bloque II)

- 25.Tiene lenguaje sexista
- 26.Uso del masculino genérico
- 27.Uso de &quot;hombre&quot; para denominar a la humanidad
- 28.Uso dual aparente (zorra/zorro)
- 29.No usa cargos, profesiones y oficios para denominar mujeres
- 30.Sexismo social en el discurso
- 31.Androcentrismo
- 32.Se mencionan mujeres sin el uso del nombre (la investigadora)
- 33.Asimetría entre mujeres y hombre en el tratamiento (nombre de pila vs. apellido)
- 34.Infantilización de las mujeres
- 35.Denominación sexualizada (chicas, niñas, mujeres...)
- 36.Denominación redundante (Una mujer ingeniera)
- 37.Denominación dependiente (Relaciones familiares de las mujeres [&quot;hija de&quot;, &quot;esposa de&quot;])
- 38.Existe criterios de excepción o noticiabilidad (la primera, la única...)
- 39.Existe comparación de mujeres a hombres

In [35]:
def cargar_variables_desde_json(ruta_archivo: str = "variables.json") -> list:
    """Carga el archivo JSON completo desde el disco."""
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"No se encontró el archivo '{ruta_archivo}'. Asegúrate de crearlo con los datos del prompt anterior.")
    except json.JSONDecodeError:
        raise ValueError(f"El archivo '{ruta_archivo}' no tiene un formato JSON válido.")

def obtener_config_variable(datos_json: list, codigo_buscado: str) -> dict:
    """Filtra la lista de variables para encontrar la configuración específica."""
    for variable in datos_json:
        if variable["codigo"] == codigo_buscado:
            return variable
    raise ValueError(f"La variable con código '{codigo_buscado}' no existe en el archivo JSON.")

def cargar_texto_template(ruta: str) -> str:
    return Path(ruta).read_text(encoding="utf-8")

def generar_prompt_dinamico(config: dict, texto: str, ruta_template: str) -> str:
    template = cargar_texto_template(ruta_template)
    
    valores = config['valores_posibles']
    
    # ESTRATEGIA: Separar la opción "No" (índice 0) de las "Sí" (índices 1 en adelante)
    # Asumimos siempre que la primera opción en tu JSON es la negativa ("No", "No se aprecia", etc.)
    opciones_activas = []
    
    # Empezamos el bucle en 1 para saltarnos el índice 0
    for i in range(1, len(valores)):
        # Código real es i+1 (porque 0 es 1, 1 es 2...)
        codigo_real = i + 1
        descripcion = valores[i]
        opciones_activas.append(f"{codigo_real} = {descripcion}")
    
    # Convertimos la lista en texto
    lista_opciones_str = "\n".join(opciones_activas)
    
    # Calculamos el rango para la instrucción JSON
    rango_codigos = f"1 al {len(valores)}"

    prompt_final = template.format(
        nombre=config['nombre'],
        definicion=config['definicion'],
        metodologia=config['metodologia'],
        ejemplos=config['ejemplos'],
        texto_input=texto,
        lista_opciones_activas=lista_opciones_str, # <--- CAMBIO: Solo opciones positivas
        rango_codigos=rango_codigos
    )
    
    return prompt_final

## 25. Tiene lenguaje sexista

In [49]:
from pathlib import Path
import json_repair

class BloqueAnalisisLenguajeSexista(BaseModel):
    """
    Modelo específico para 'lenguaje_sexista' que tiene 3 valores:
    1 = No
    2 = Sí
    3 = Sí; además se observa un salto semántico
    """
    codigo: int = Field(
        ..., 
        ge=1, 
        le=3, 
        description="Selección numérica: 1='No', 2='Sí', 3='Sí; además se observa un salto semántico'"
    )
    explicacion: str = Field(
        ..., 
        description="Cadena de pensamiento (Chain of Thought). Explica paso a paso por qué se ha seleccionado ese código."
    )
    evidencias: List[str] = Field(
        ..., 
        description="Lista exacta de frases, palabras o fragmentos extraídos del texto que justifican la decisión."
    )


def clasificar_var_lenguaje_sexista(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "25") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [50]:
# --- Uso ---
lenguaje_sexista = clasificar_var_lenguaje_sexista(articulo.text)
print(f"lenguaje_sexista codigo: {lenguaje_sexista.codigo}")
print(f"lenguaje_sexista evidencias: {lenguaje_sexista.evidencias}")
print(f"lenguaje_sexista explicacion: {lenguaje_sexista.explicacion}")

--- Analizando lenguaje_sexista con template dinámico ---
lenguaje_sexista codigo: 2
lenguaje_sexista evidencias: ['‘mujeres’ como modelos de paz y unidad', '‘madres’ y ‘mujeres’ como ‘reunión de la familia humana’', '‘expareja’ como perpetrador de la violencia']
lenguaje_sexista explicacion: El texto, a pesar de tratar un tema crucial como la violencia contra las mujeres, perpetúa un sesgo de género inherente a la forma en que se construye la narrativa. La constante referencia a ‘madres’ y ‘mujeres’ como modelos de paz, unidad y ‘reunión de la familia humana’ refuerza la idea de que las mujeres deben ser vistas principalmente por su rol maternal y su capacidad para ‘construir’ y ‘reunir’.  La repetición de estas categorías, sin una exploración de las diversas formas en que las mujeres pueden ejercer poder y liderazgo, reproduce un estereotipo. Además, la mención de ‘expareja’ en el contexto de la violencia, aunque relevante, se centra en la figura masculina como perpetrador, sin una e

## 26. Uso del masculino genérico

In [ ]:
class BloqueAnalisisBinario(BaseModel):
    """
    Modelo para variables con respuesta binaria:
    1 = No
    2 = Sí
    """
    codigo: int = Field(
        ..., 
        ge=1, 
        le=2, 
        description="Selección numérica: 1='No', 2='Sí'"
    )
    explicacion: str = Field(
        ..., 
        description="Cadena de pensamiento (Chain of Thought). Explica paso a paso por qué se ha seleccionado ese código, aplicando la metodología definida."
    )
    evidencias: List[str] = Field(
        ..., 
        description="Lista exacta de frases, palabras o fragmentos extraídos del texto que justifican la decisión."
    )




def clasificar_var_masc_generico(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "26") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [52]:
# --- Uso ---
masc_generico = clasificar_var_masc_generico(articulo.text)
print(f"masc_generico codigo: {masc_generico.codigo}")
print(f"masc_generico evidencias: {masc_generico.evidencias}")
print(f"masc_generico explicacion: {masc_generico.explicacion}")

--- Analizando masc_generico con template dinámico ---
masc_generico codigo: 2
masc_generico evidencias: ['‘el hombre’']
masc_generico explicacion: El texto exhibe un uso sistemático del género gramatical masculino como referente universal ('el papa Francisco', 'el mundo necesita mirar a las madres', 'el hombre'). Esta naturalización del masculino como norma y la invisibilización de la mujer a través de la referencia constante al masculino, constituye un claro ejemplo de la variable 'masc_generico'. La construcción de la realidad se centra en la figura masculina, perpetuando un androcentrismo implícito.


## 27. Uso de &quot;hombre&quot; para denominar a la humanidad

In [53]:
def clasificar_var_hombre_denominar_humanidad(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "27") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [54]:
# --- Uso ---
clasificar_var_hombre_denominar_humanidad = clasificar_var_hombre_denominar_humanidad(articulo.text)
print(f"clasificar_var_hombre_denominar_humanidad codigo: {clasificar_var_hombre_denominar_humanidad.codigo}")
print(f"clasificar_var_hombre_denominar_humanidad evidencias: {clasificar_var_hombre_denominar_humanidad.evidencias}")
print(f"clasificar_var_hombre_denominar_humanidad explicacion: {clasificar_var_hombre_denominar_humanidad.explicacion}")

--- Analizando hombre_denominar_humanidad con template dinámico ---
clasificar_var_hombre_denominar_humanidad codigo: 2
clasificar_var_hombre_denominar_humanidad evidencias: ['‘el hombre de la inteligencia artificial’']
clasificar_var_hombre_denominar_humanidad explicacion: El texto exhibe consistentemente la variable ‘hombre_denominar_humanidad’ a través de la constante referencia a ‘el hombre’ como referente universal, incluso cuando se habla de la humanidad en general. Frases como ‘quien lastima a una mujer profana a Dios, nacido de mujer’ y ‘necesita de una Madre que vuelva a reunir a la familia humana’ establecen al hombre como el modelo de humanidad, invisibilizando la mujer y asumiendo que la humanidad se define a través de la figura masculina. La repetición de ‘el hombre’ en contextos que deberían ser inclusivos, como ‘el hombre de la calle’ o ‘el hombre de la inteligencia artificial’, refuerza este sesgo.


## 28. Uso dual aparente (zorro/zorra)

In [55]:
def clasificar_var_uso_dual_zorr(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "28") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [56]:
# --- Uso ---
uso_dual_zorr = clasificar_var_uso_dual_zorr(articulo.text)
print(f"uso_dual_zorr codigo: {uso_dual_zorr.codigo}")
print(f"uso_dual_zorr evidencias: {uso_dual_zorr.evidencias}")
print(f"uso_dual_zorr explicacion: {uso_dual_zorr.explicacion}")

--- Analizando uso_dual_zorr con template dinámico ---
uso_dual_zorr codigo: 2
uso_dual_zorr evidencias: ['‘Miremos a María para ser constructores de unidad’', '‘La inteligencia artificial debe ser entendida como una galaxia de realidades distintas’', '‘La Iglesia necesita de María para redescubrir su propio rostro femenino’']
uso_dual_zorr explicacion: El texto exhibe un uso sistemático de la variable ‘uso_dual_zorr’ a través de la constante referencia a ‘mujer’ y ‘hombre’ como categorías fundamentales para describir roles, responsabilidades y cualidades. La reiterada asociación de ‘mujer’ con atributos positivos como ‘cuidado’, ‘solicitud’, ‘generativa’, ‘madre’, ‘constructora de unidad’ y ‘emancipación’ en contraposición a la ‘cultura machista’ que genera ‘violencia’ y ‘odio’, revela una inversión de valores y una construcción de realidad que privilegia la figura femenina como modelo y solución, perpetuando así el androcentrismo. La constante referencia a la ‘mujer’ como ‘don’ y la 

## 29. No usa cargos, profesiones y oficios para denominar mujeres

In [57]:
def clasificar_var_uso_cargo_mujer(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "29") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [58]:
# --- Uso ---
uso_cargo_mujer = clasificar_var_uso_cargo_mujer(articulo.text)
print(f"uso_cargo_mujer codigo: {uso_cargo_mujer.codigo}")
print(f"uso_cargo_mujer evidencias: {uso_cargo_mujer.evidencias}")
print(f"uso_cargo_mujer explicacion: {uso_cargo_mujer.explicacion}")

--- Analizando uso_cargo_mujer con template dinámico ---
uso_cargo_mujer codigo: 2
uso_cargo_mujer evidencias: ["'subsecretaria del dicasterio'", "'puestos de dirección'", "'la religiosa italiana Alessandra Smerilli fue nombrada'"]
uso_cargo_mujer explicacion: El texto exhibe de forma recurrente el 'uso_cargo_mujer' a través de la utilización constante del masculino gramatical para referirse a mujeres en roles de liderazgo y autoridad ('subsecretaria del dicasterio', 'puestos de dirección', 'la religiosa italiana').  Aunque el texto intenta transmitir una imagen de progreso y cambio, la naturalización del masculino como norma y la falta de especificidad en la denominación de las mujeres perpetúan la invisibilización y el androcentrismo. La frase 'la religiosa italiana Alessandra Smerilli fue nombrada' es particularmente reveladora, ya que no se nombra a la mujer directamente, sino que se la define a través de su cargo.


## 30. Sexismo social en el discurso

In [59]:
def clasificar_var_sexismo_discurso(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "30") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [60]:
# --- Uso ---
sexismo_discurso = clasificar_var_sexismo_discurso(articulo.text)
print(f"sexismo_discurso codigo: {sexismo_discurso.codigo}")
print(f"sexismo_discurso evidencias: {sexismo_discurso.evidencias}")
print(f"sexismo_discurso explicacion: {sexismo_discurso.explicacion}")

--- Analizando sexismo_discurso con template dinámico ---
sexismo_discurso codigo: 2
sexismo_discurso evidencias: ['‘Mujer’ – se utiliza repetidamente para referirse a las víctimas de violencia y a la solución a los problemas.', "'redescubrir su rostro femenino' – implica que la mujer necesita ser ‘redescubierta’ como modelo ideal.", "'Figura Perfecta' – María es presentada como un modelo inalcanzable y desvaloriza las características y experiencias de las mujeres en el presente."]
sexismo_discurso explicacion: El texto, a pesar de denunciar la violencia contra las mujeres y reconocer avances en la participación femenina en la Iglesia, perpetúa un sexismo discursivo al idealizar la mujer como ‘Madre’, ‘Constructora de Unidad’ y ‘Figura Perfecta’ – roles tradicionales y asimétricos. La constante referencia a la ‘Madre’ y a la ‘Mujer’ como solución a problemas globales (paz, unidad) refuerza la idea de que la responsabilidad de la resolución de conflictos recae, inherentemente, en el gén

## 31. Androcentrismo

In [61]:
def clasificar_var_androcentrismo(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "31") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [62]:
# --- Uso ---
androcentrismo = clasificar_var_androcentrismo(articulo.text)
print(f"androcentrismo codigo: {androcentrismo.codigo}")
print(f"androcentrismo evidencias: {androcentrismo.evidencias}")
print(f"androcentrismo explicacion: {androcentrismo.explicacion}")

--- Analizando androcentrismo con template dinámico ---
androcentrismo codigo: 2
androcentrismo evidencias: ['‘La Iglesia necesita de María para redescubrir su propio rostro femenino’', '‘La Iglesia necesita de María para redescubrir su propio rostro femenino’', '‘La señora de Karembeu’ (aunque no explícitamente, la referencia a ‘la señora’ es un ejemplo de uso de un referente femenino como punto de referencia, implicando una jerarquía’']
androcentrismo explicacion: El texto exhibe un androcentrismo evidente a través de la constante referencia a la ‘Madre’ (María) como modelo y solución a problemas sociales y existenciales. La construcción de la ‘Madre’ como figura central, y la invocación de su ‘dón’ (regalo) de ‘cuidado y solicitud’ y ‘paciencia y valentía materna’ refuerza la idea de que la mujer debe ser vista y valorada principalmente por sus cualidades maternales y su capacidad para ‘cuidar’.  Además, la reiterada mención de ‘la Iglesia necesita de María’ implica que la solución 

## 32. Se mencionan mujeres sin el uso del nombre (la investigadora)

In [64]:
def clasificar_var_mencion_nombre_investigadora(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "32") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [65]:
# --- Uso ---
mencion_nombre_investigadora = clasificar_var_mencion_nombre_investigadora(articulo.text)
print(f"mencion_nombre_investigadora codigo: {mencion_nombre_investigadora.codigo}")
print(f"mencion_nombre_investigadora evidencias: {mencion_nombre_investigadora.evidencias}")
print(f"mencion_nombre_investigadora explicacion: {mencion_nombre_investigadora.explicacion}")

--- Analizando mencion_nombre_investigadora con template dinámico ---
mencion_nombre_investigadora codigo: 1
mencion_nombre_investigadora evidencias: []
mencion_nombre_investigadora explicacion: El texto, en su conjunto, no presenta evidencia clara de la variable 'mencion_nombre_investradora'. Si bien se menciona a figuras como Francisco y Alessandra Smerilli, la mención se realiza de forma genérica, utilizando títulos y cargos en lugar de nombres y apellidos. La redacción es descriptiva y enfocada en el rol y la función, lo cual es común en el periodismo, pero no indica una invisibilización intencionada o un sesgo de género en la forma de identificar a las figuras femeninas. La repetición de 'el Papa Francisco' refuerza esta tendencia.


## 33. Asimetría entre mujeres y hombre en el tratamiento (nombre de pila vs. apellido)

In [66]:
def clasificar_var_asimetria_mujer_hombre(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "33") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [67]:
# --- Uso ---
asimetria_mujer_hombre = clasificar_var_asimetria_mujer_hombre(articulo.text)
print(f"asimetria_mujer_hombre codigo: {asimetria_mujer_hombre.codigo}")
print(f"asimetria_mujer_hombre evidencias: {asimetria_mujer_hombre.evidencias}")
print(f"asimetria_mujer_hombre explicacion: {asimetria_mujer_hombre.explicacion}")

--- Analizando asimetria_mujer_hombre con template dinámico ---
asimetria_mujer_hombre codigo: 2
asimetria_mujer_hombre evidencias: ['‘el papa Francisco’', '‘Alessandra Smerilli’', '‘María’', '‘la religiosa italiana Alessandra Smerilli’']
asimetria_mujer_hombre explicacion: El texto exhibe una asimetría de tratamiento nominal significativa. Se refiere a Francisco como ‘el papa Francisco’ repetidamente, utilizando su título y nombre de pila, mientras que las mujeres, incluso figuras religiosas importantes como María y Alessandra Smerilli, se presentan con sus apellidos y nombres completos. Esta práctica, aunque aparentemente neutral, refuerza la construcción de una jerarquía de género, situando al hombre en la posición central y de referencia, y relegando a las mujeres a un segundo plano a través de la formalización del tratamiento. La repetición del tratamiento formal para Francisco, en contraposición al tratamiento más informal y personal para las mujeres, es una manifestación sutil p

## 34. Infatilización de las mujeres

In [69]:
def clasificar_var_disminutivos_infantilizacion(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "34") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [70]:
# --- Uso ---
disminutivos_infantilizacion = clasificar_var_disminutivos_infantilizacion(articulo.text)
print(f"disminutivos_infantilizacion codigo: {disminutivos_infantilizacion.codigo}")
print(f"disminutivos_infantilizacion evidencias: {disminutivos_infantilizacion.evidencias}")
print(f"disminutivos_infantilizacion explicacion: {disminutivos_infantilizacion.explicacion}")

--- Analizando disminutivos_infantilizacion con template dinámico ---
disminutivos_infantilizacion codigo: 2
disminutivos_infantilizacion evidencias: ['‘cuidado y solicitud’']
disminutivos_infantilizacion explicacion: El texto exhibe una fuerte infantilización de la mujer a través de la constante referencia a la ‘Madre’, ‘galaxia de realidades distintas’ (en referencia a la IA), y la reiterada invocación de ‘cuidado y solicitud’, ‘paciencia y valentía materna’. Estas expresiones, aunque aparentemente positivas en su intención, operan como un mecanismo de naturalización de roles tradicionales y una representación de la mujer como un ser inherentemente protector y dependiente, reforzando la imagen de ‘Madre’ como modelo a seguir y limitando su autonomía. La repetición de términos asociados a la infancia – ‘Madre’, ‘cuidado’ – es un indicativo claro de esta infantilización. La referencia a ‘C’è ancora domani’ como herramienta didáctica también puede interpretarse como una idealización de 

## 35. Denominación sexualizada (chicas, niñas, mujeres...)

In [72]:
def clasificar_var_denominacion_sexualizada(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "35") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [73]:
# --- Uso ---
denominacion_sexualizada = clasificar_var_denominacion_sexualizada(articulo.text)
print(f"denominacion_sexualizada codigo: {denominacion_sexualizada.codigo}")
print(f"denominacion_sexualizada evidencias: {denominacion_sexualizada.evidencias}")
print(f"denominacion_sexualizada explicacion: {denominacion_sexualizada.explicacion}")

--- Analizando denominacion_sexualizada con template dinámico ---
denominacion_sexualizada codigo: 2
denominacion_sexualizada evidencias: ['‘La Iglesia necesita de María’', '‘La Iglesia necesita de María para redescubrir su propio rostro femenino’', '‘La Iglesia necesita de María para ser generativa a través de una pastoral hecha de cuidado y solicitud, de paciencia y valentía materna’', '‘La Iglesia necesita de María para ser constructores de unidad’']
denominacion_sexualizada explicacion: El texto, a pesar de tratar un tema urgente como la violencia contra las mujeres, perpetúa la variable ‘denominación sexualizada’ al referirse constantemente a las mujeres como ‘Madre’, ‘Mujeres’, ‘Familia’ y ‘la Iglesia necesita de María’.  Esta recurrencia, aunque intencionalmente para evocar un rol tradicional y protector, reduce a las mujeres a su función biológica y social, despojándolas de su individualidad y agencia. La insistencia en ‘María’ como modelo, aunque busca empoderamiento, sigue si

## 36. Denominación redundante (Una mujer ingeniera)

In [75]:
def clasificar_var_denominacion_redundante(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "36") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [77]:
# --- Uso ---
denominacion_redundante = clasificar_var_denominacion_redundante(articulo.text)
print(f"denominacion_redundante codigo: {denominacion_redundante.codigo}")
print(f"denominacion_redundante evidencias: {denominacion_redundante.evidencias}")
print(f"denominacion_redundante explicacion: {denominacion_redundante.explicacion}")

--- Analizando denominacion_redundante con template dinámico ---
denominacion_redundante codigo: 2
denominacion_redundante evidencias: ['‘el papa Francisco’', '‘el Pontífice’', '‘el Papa publica algunas semanas antes un mensaje especial’']
denominacion_redundante explicacion: El texto, a pesar de tratar un tema urgente como la violencia contra las mujeres, perpetúa la variable ‘denominación_redundante’ al referirse constantemente a ‘el papa Francisco’ y ‘el Pontífice’.  La repetición de títulos honoríficos que ya implican la identidad del líder religioso, en lugar de utilizar simplemente ‘Francisco’, refuerza una visión patriarcal donde la figura masculina es siempre la principal y la que da nombre a las acciones y referencias.  La constante utilización de ‘el Pontífice’  es un ejemplo claro de cómo el lenguaje puede naturalizar la autoridad masculina y relegar a la mujer a un rol secundario, incluso cuando se aborda un problema de desigualdad.


## 37. Denominación dependiente (Relaciones familiares de las mujeres [&quot;hija de&quot;, &quot;esposa de&quot;])

In [79]:
def clasificar_var_denominacion_dependiente(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "37") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [80]:
# --- Uso ---
denominacion_dependiente = clasificar_var_denominacion_dependiente(articulo.text)
print(f"denominacion_dependiente codigo: {denominacion_dependiente.codigo}")
print(f"denominacion_dependiente evidencias: {denominacion_dependiente.evidencias}")
print(f"denominacion_dependiente explicacion: {denominacion_dependiente.explicacion}")

--- Analizando denominacion_dependiente con template dinámico ---
denominacion_dependiente codigo: 2
denominacion_dependiente evidencias: ['‘la hija de Ali’', '‘la ex de Iker Casillas’', '‘la señora de Karembeu’', '‘María’ (mención constante como modelo y figura idealizada)']
denominacion_dependiente explicacion: El texto, a pesar de su intento de abordar la violencia de género y el papel de las mujeres en la Iglesia, recurre consistentemente a la denominación dependiente. La referencia a ‘la hija de Ali’, ‘la ex de Iker Casillas’, ‘la señora de Karembeu’ y la reiterada mención a ‘María’ como modelo, establecen una relación de las mujeres a través de su vínculo con hombres. Esta construcción refuerza la idea de que la identidad femenina está definida en relación con figuras masculinas, perpetuando un androcentrismo implícito. La constante referencia a ‘María’ como ‘modelo’ es particularmente problemática, ya que la idealiza como una figura femenina sin agencia propia, reduciendo su val

## 38. Existe criterios de excepción o noticiabilidad (la primera, la única...)

In [82]:
def clasificar_var_criterios_excepcion(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "38") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [83]:
# --- Uso ---
criterios_excepcion = clasificar_var_criterios_excepcion(articulo.text)
print(f"criterios_excepcion codigo: {criterios_excepcion.codigo}")
print(f"criterios_excepcion evidencias: {criterios_excepcion.evidencias}")
print(f"criterios_excepcion explicacion: {criterios_excepcion.explicacion}")

--- Analizando criterios_excepcion con template dinámico ---
criterios_excepcion codigo: 2
criterios_excepcion evidencias: ['‘figuras perfectas’']
criterios_excepcion explicacion: El texto, a pesar de abordar la violencia contra las mujeres y los esfuerzos de la Iglesia para promover la igualdad, perpetúa el criterio de excepción al presentar a las mujeres como ‘recolección de dones’ y ‘figuras perfectas’ a través de la analogía con María. La referencia a ‘redescubrir su rostro femenino’ y ‘dar espacio a las mujeres’ implica que su rol es excepcional y necesita ser activamente buscado y restaurado, en lugar de ser considerado intrínseco a la Iglesia. Se establece una diferencia entre el rol ‘natural’ de la mujer y el rol que la Iglesia ‘debe’ darle, reforzando la idea de que la mujer es una anomalía que necesita ser reconocida y valorada. La mención de la subsecretaria como ‘el cargo más alto ocupado por una mujer’ también refuerza esta percepción de excepcionalidad.


## 39. Existe comparación de mujeres a hombres

In [85]:
def clasificar_var_comparacion_mujer_hombre(
    texto_articulo: str, 
    ruta_json: str = "variables.json",
    ruta_template: str = "prompts/prompt_clara.md"
) -> BloqueAnalisisLenguajeSexista:
    
    # A. Carga Configuración
    try:
        vars_data = cargar_variables_desde_json(ruta_json)
        config = obtener_config_variable(vars_data, "39") 
    except Exception as e:
        return BloqueAnalisisLenguajeSexista(codigo=1, explicacion=f"Error config: {e}", evidencias=[])

    # ✅ CAMBIO 2: Limpieza preventiva del texto ANTES del prompt
    # Si el artículo tiene comillas dobles ("), las convertimos a simples (')
    # para que no confundan al modelo al generar el JSON.
    texto_seguro = texto_articulo.replace('"', "'")

    # B. Generación del Prompt (pasamos el texto seguro)
    prompt = generar_prompt_dinamico(config, texto_seguro, ruta_template)

    # C. Llamada a Ollama
    print(f"--- Analizando {config['nombre']} con template dinámico ---")
    respuesta_raw = consultar_ollama(prompt, temperature=0.1) 

    # D. Parsing y Validación ROBUSTA
    try:
        # ✅ CAMBIO 3: Usar json_repair en lugar de json.loads
        # json_repair hace tres cosas automáticamente:
        # 1. Quita los bloques Markdown (```json)
        # 2. Arregla comillas dobles rotas dentro de los textos
        # 3. Cierra llaves faltantes
        data = json_repair.loads(respuesta_raw)
        
        return BloqueAnalisisLenguajeSexista(**data)

    except ValidationError as e:
        print(f"⚠️ Error de validación Pydantic: {e}")
        # Retorno de seguridad si faltan campos obligatorios
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"El modelo no devolvió los campos correctos. Raw: {str(e)}", 
            evidencias=[]
        )
    except Exception as e:
        print(f"❌ Error fatal: {e}")
        return BloqueAnalisisLenguajeSexista(
            codigo=1, 
            explicacion=f"Error técnico irrecuperable: {str(e)}", 
            evidencias=[]
        )

In [86]:
# --- Uso ---
comparacion_mujer_hombre = clasificar_var_comparacion_mujer_hombre(articulo.text)
print(f"comparacion_mujer_hombre codigo: {comparacion_mujer_hombre.codigo}")
print(f"comparacion_mujer_hombre evidencias: {comparacion_mujer_hombre.evidencias}")
print(f"comparacion_mujer_hombre explicacion: {comparacion_mujer_hombre.explicacion}")

--- Analizando comparacion_mujer_hombre con template dinámico ---
comparacion_mujer_hombre codigo: 2
comparacion_mujer_hombre evidencias: ['‘Miremos a María para ser constructores de unidad’', '‘La mujer… representa su modelo y su figura perfecta’', '‘La inteligencia artificial debe ser entendida como una galaxia de realidades distintas’ (implica una valoración de la ‘realidad’ masculina como norma y la femenina como unida a ella)']
comparacion_mujer_hombre explicacion: El texto, a pesar de su intento de abordar la violencia contra las mujeres y la necesidad de empoderamiento femenino en la Iglesia, perpetúa la variable ‘mujer-hombre’ a través de la constante referencia a la ‘mujer’ como modelo a imitar y como fuente de ‘paz’ y ‘unidad’. La frase ‘Miremos a María para ser constructores de unidad’ es particularmente reveladora, ya que establece a la mujer (María) como el referente ideal, un modelo a seguir, lo que refuerza la idea de una jerarquía de género donde la figura masculina es 

---